# OTMapper test on WD for Conference Properties
Improveing OTMapper precision, recall, and scalability by
1. ranking candidates by WD distance between concept with its context
2. applying OT on mutual NN
3. non-uniform distributions
4. applying OT On blocks of ontologies by LSH
5. others

In [1]:
import OTMapOnto as maponto
import OTMapper as otmapper
import OTNeighborhood_TDA as mapneighbor
from rdflib import Graph

import pandas as pd
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt

import ot

import importlib

from collections import defaultdict

from tqdm import tqdm

import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

INFO:root:logging info test


In [2]:
%%time
model_path="../model/crawl-300d-2M-subword.bin"
embs_model = maponto.load_embeddings(model_path, None)

INFO:root:load_embeddings(pretrained_path, model): Load pre-trained embeddings for about 10 mins if the model hasn't been load yet.
INFO:root:load_embeddings(pretrained_path, model): Loading the pre-trained Fasttext model...Please be patient...It may take about 10 mins to load...
INFO:gensim.models._fasttext_bin:loading 2000000 words for fastText model from ../model/crawl-300d-2M-subword.bin


Loading the pre-trained Fasttext model...Please be patient...It may take about 10 mins to load...


ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'DeutschHrvatskiEnglishDanskNederlandssuomiFran\xc3\xa7ais\xce\x95\xce\xbb\xce\xbb\xce'; replacing invalid characters, using 'DeutschHrvatskiEnglishDanskNederlandssuomiFrançaisΕλλ\\xce'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xe3\x81\x99\xe3\x81\xb9\xe3\x81\xa6\xe3\x81\xae\xe5\x9b\x9e\xe7\xad\x94\xe3\x82\x92\xe9\x9d\x9e\xe8\xa1\xa8\xe7\xa4\xba\xe3\x81\xab\xe3\x81\x99\xe3\x82\x8b\xe8\xb3\xaa\xe5\x95\x8f\xe3\x82\x92\xe5\x89\x8a\xe9\x99\xa4\xe3\x81\x97\xe3'; replacing invalid characters, using 'すべての回答を非表示にする質問を削除し\\xe3'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'00Z\xe9\x83\xa8\xe5\xb1\x8b\xe3\x82\xbf\xe3\x82\xa4\xe3\x83\x97\xe3\x81\xbe\xe3\x82\x8b\xe3\x81\xbe\xe3\x82\x8b\xe8\xb2\xb8\xe5\x88\x87\xe5\xbb\xba\xe7\x89\xa9\xe3\x82\xbf\xe3\x82\xa4\xe3\x83\x97\xe4\xb8\x80\xe8\xbb\x92\xe5'; replacing invalid characters, using '00Z部屋タイプまるまる貸切建物タイプ一軒\\xe5'
E

Completed loading the pre-trained Fasttext model.
CPU times: user 3min 33s, sys: 19.7 s, total: 3min 53s
Wall time: 5min 28s


In [3]:
mapper = otmapper.Mapper(embs_model)

In [4]:
importlib.reload(maponto)
importlib.reload(mapneighbor)
importlib.reload(otmapper)

INFO:root:logging info test


<module 'OTMapper' from '/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapper.py'>

# Class Matching - Refined by Label Syn Lists Jaccard

In [92]:
import pickle
with open("../data/conference/with_references/preds_candidates_list.pk", "rb") as file:
    preds_candidates_list = pickle.load(file)

In [123]:
len(preds_candidates_list)

21

In [124]:
preds_candidates_list[20].head()

,source,source_label_x,target,target_label_x,target_label_y,source_label_y,string_similarity,context_similarity,label_embeddings_similarity,label_embeddings_wd_similarity,string_context_similarity,label_emb_context_similarity,label_emb_wd_context_similarity,label_emb_wd_context_all_similarity
0,http://conference#Abstract,Abstract,http://ekaw#Abstract,Abstract,Abstract,Abstract,1.0,0.938063,1.0,1.0,0.938063,0.938063,0.938063,0.938063
1,http://conference#Conference,Conference,http://ekaw#Conference,Conference,Conference,Conference,1.0,0.795362,1.0,1.0,0.795362,0.795362,0.795362,0.795362
2,http://conference#Conference_participant,Conference_participant,http://ekaw#Conference_Participant,Conference_Participant,Conference_Participant,Conference_participant,1.0,0.888514,1.0,1.0,0.888514,0.888514,0.888514,0.888514
3,http://conference#Conference_proceedings,Conference_proceedings,http://ekaw#Conference_Proceedings,Conference_Proceedings,Conference_Proceedings,Conference_proceedings,1.0,0.653297,1.0,1.0,0.653297,0.653297,0.653297,0.653297
4,http://conference#Invited_speaker,Invited_speaker,http://ekaw#Invited_Speaker,Invited_Speaker,Invited_Speaker,Invited_speaker,1.0,0.967539,1.0,1.0,0.967539,0.967539,0.967539,0.967539


In [155]:
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)
all_refs = []
for subf in subfolders:
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(refs_url):
        all_refs.append(refs_url)

In [7]:
# tune refinement
def tune_refine_matchings_synList(thre):
    evaluations = []
    refined_candidates_list = []

    for k, candidates in enumerate(preds_candidates_list):
        #if k > 0:
        #    break
        idxs = []
        jacSims = []
        sources = []
        targets = []
        print(candidates.shape)
        for i, row in tqdm(candidates.iterrows()):
            source_uri = row['source']
            target_uri = row['target']
            if (source_uri in sources) or (target_uri in targets):
                pass
            else:
                if (row['string_similarity'] > 0.95):
                    idxs.append(i)
                    jacSims.append(row['string_similarity'])
                    sources.append(source_uri)
                    targets.append(target_uri)
                    #print(source_uri, "|", target_uri)
                else:
                    # compute the Jaccard similarity
                    slabel = maponto.clean_extract_label_from_uri(source_uri).strip()
                    tlabel = maponto.clean_extract_label_from_uri(target_uri).strip()

                    slabel_syn_lists = mapneighbor.get_syn_lists(slabel, 5)
                    tlabel_syn_lists = mapneighbor.get_syn_lists(tlabel, 5)

                    size_union = len(slabel_syn_lists) +  len(tlabel_syn_lists)
                    size_intersection = 0
                    tmapped = []
                    for sList in slabel_syn_lists:
                        #print('sList', sList)
                        for j, tList in enumerate(tlabel_syn_lists):
                            if (j not in tmapped) & (len(set(sList).intersection(set(tList))) > 0):
                                #print('sList', sList)
                                #print('tList', tList)
                                size_intersection = size_intersection + 1
                                size_union = size_union - 1
                                tmapped.append(j)
                                break

                    jacSim = size_intersection / size_union

                    if (jacSim >= thre):
                        idxs.append(i)
                        jacSims.append(jacSim)
                        sources.append(source_uri)
                        targets.append(target_uri)
                        #print(source_uri, "|", target_uri)

        #print(idxs)
        refined_candidates = candidates.loc[idxs].reset_index(drop=True)
        refined_candidates['Jaccard_similarity'] = jacSims
        refined_candidates_list.append(refined_candidates)

        evaluation = maponto.evaluate_noprint(refined_candidates, all_refs[k])

        evaluations.append(evaluation)

    return refined_candidates_list, pd.DataFrame(evaluations)

In [176]:
thre_refined_candidates_list = {}
thre_evaluations_df = {}
for thre in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    refined_candidates_list, evaluations_df = tune_refine_matchings_synList(thre)
    thre_refined_candidates_list[thre] = refined_candidates_list
    thre_evaluations_df[thre] = evaluations_df

(2579, 14)


2579it [00:25, 99.46it/s] 


(949, 14)


949it [00:07, 135.16it/s]


(1871, 14)


...

1940it [00:19, 99.38it/s] 


(1908, 14)


1908it [00:21, 87.35it/s] 


(1274, 14)


1274it [00:10, 126.16it/s]


In [165]:
len(refined_candidates_list)

21

In [177]:
with open("../data/conference/with_references/threshold_2_refined_candidates_synList_list.pk", "wb") as file:
    pickle.dump(thre_refined_candidates_list, file)

In [178]:
with open("../data/conference/with_references/threshold_2_refined_candidates_synList_evaluations_df.pk", "wb") as file:
    pickle.dump(thre_evaluations_df, file)

In [166]:
#with open("../data/conference/with_references/refined_class_candidates_synLists_list.pk", "wb") as file:
#    pickle.dump(refined_candidates_list, file)

In [192]:
thre_evaluations_df[0.65].mean()

numOfRefs                  14.523810
numOfCorrectlyPredicted     7.238095
numOfPredicted              8.952381
precision                   0.823759
recall                      0.530648
f1                          0.633690
dtype: float64

In [193]:
thre_evaluations_df[0.7].mean()

numOfRefs                  14.523810
numOfCorrectlyPredicted     7.047619
numOfPredicted              8.619048
precision                   0.831960
recall                      0.518426
f1                          0.627635
dtype: float64

# Exact Property Matchings for Individual Conference Cases+ Jaccard on Sets 0.80

In [194]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synListJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.80, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synListJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.80, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245
1. #sobject=(70, 2), #tobject=(15, 2)


70it [00:06, 10.57it/s]


1. #sdatatype=(3, 2), #tdatatype=(11, 2)


3it [00:00, 12.84it/s]


../data/conference/with_references/confof-sigkdd/source.rdf
../data/conference/with_references/confof-sigkdd/target.rdf
2. len(source_graph)=539, len(target_graph)=245
2. #sobject=(15, 2), #tobject=(15, 2)


...

20. #sdatatype=(11, 2), #tdatatype=(21, 2)


11it [00:02,  4.47it/s]


../data/conference/with_references/conference-ekaw/source.rdf
../data/conference/with_references/conference-ekaw/target.rdf
21. len(source_graph)=639, len(target_graph)=557
21. #sobject=(54, 2), #tobject=(33, 2)


54it [00:18,  2.93it/s]

21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 5min 26s, sys: 17.4 s, total: 5min 44s
Wall time: 5min 43s


In [195]:
len(all_refined_object_aligns_list)

21

In [212]:
with open("../data/conference/with_references/all_refined_object_aligns_jac0.8_list.pk", "wb") as file:
    pickle.dump(all_refined_object_aligns_list, file)

In [227]:
len(all_refined_datatype_aligns_list)

21

In [213]:
with open("../data/conference/with_references/all_refined_datatype_aligns_jac0.8_list.pk", "wb") as file:
    pickle.dump(all_refined_datatype_aligns_list, file)

In [208]:
pd.concat(all_refined_object_aligns_list)

,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://cmt#submitPaper,http://sigkdd#submit,cmt-sigkdd,submitPaper,submit,Author submitPaper Paper,Author submit Paper,1.0,0.8


In [209]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_99446/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  12.000000
numOfCorrectlyPredicted     1.000000
numOfPredicted              1.000000
precision                   1.000000
recall                      0.083333
f1                          0.153846
dtype: float64

In [229]:
pd.concat(all_refined_datatype_aligns_list)

,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://conference#has_a_name,http://sigkdd#Name_of_conference,conference-sigkdd,has_a_name,Name_of_conference,Conference has_a_name,Conference Name_of_conference,1.0,0.8
0,http://cmt#email,http://conference#has_an_email,cmt-conference,email,has_an_email,Person email,Person has_an_email,1.0,0.8
1,http://cmt#name,http://conference#has_a_name,cmt-conference,name,has_a_name,Conference name,Conference has_a_name,1.0,0.8
0,http://cmt#email,http://confOf#hasEmail,cmt-confof,email,hasEmail,Person email,Person hasEmail,1.0,0.8
0,http://cmt#name,http://sigkdd#Name_of_conference,cmt-sigkdd,name,Name_of_conference,Conference name,Conference Name_of_conference,1.0,0.8
0,http://confOf#hasEmail,http://edas#hasEmail,confof-edas,hasEmail,hasEmail,Person hasEmail,Person hasEmail,1.0,0.8
1,http://confOf#hasFirstName,http://edas#hasFirstName,confof-edas,hasFirstName,hasFirstName,Person hasFirstName,Person hasFirstName,1.0,0.8
0,http://conference#has_an_email,http://confOf#hasEmail,conference-confof,has_an_email,hasEmail,Person has_an_email,Person hasEmail,1.0,0.8
1,http://conference#has_the_first_name,http://confOf#hasFirstName,conference-confof,has_the_first_name,hasFirstName,Person has_the_first_name,Person hasFirstName,1.0,0.8
0,http://edas#hasName,http://sigkdd#Name_of_conference,edas-sigkdd,hasName,Name_of_conference,Conference hasName,Conference Name_of_conference,1.0,0.8


In [211]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_99446/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  15.222222
numOfCorrectlyPredicted     0.888889
numOfPredicted              1.777778
precision                   0.500000
recall                      0.055494
f1                          0.097923
dtype: float64

# Combine Class Matching (Refined by Label Syn Lists Jaccard) with Exact Property Matchings (Jaccard on Sets 0.8)

In [274]:
refined_candidates_list = thre_refined_candidates_list[0.6]

In [275]:
all_evaluations = []
for i, candidates in enumerate(refined_candidates_list):
    #print(i)
    aligns_class = candidates[['source', 'target']]
    
    aligns_object = None
    if all_refined_object_aligns_list[i] is not None:
        aligns_object = all_refined_object_aligns_list[i][['source', 'target']]
        
    aligns_datatype = None
    if all_refined_datatype_aligns_list[i] is not None:
        aligns_datatype = all_refined_datatype_aligns_list[i][['source', 'target']]
        
    aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
    evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, all_refs[i])
    evaluation_aligns_class_property['test_case']= all_refs[i][35:-14]
    all_evaluations.append(evaluation_aligns_class_property)

In [277]:
pd.DataFrame(all_evaluations).mean()

/tmp/ipykernel_99446/3898145320.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.DataFrame(all_evaluations).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.666667
numOfPredicted              9.761905
precision                   0.800533
recall                      0.558399
f1                          0.647275
dtype: float64

# Inspecting Matching Examples

In [302]:
pd.DataFrame(all_evaluations)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,14,0.857143,0.800000,0.827586,iasted-sigkdd
1,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd
2,23,10,14,0.714286,0.434783,0.540541,edas-ekaw
3,15,8,11,0.727273,0.533333,0.615385,conference-sigkdd
4,19,8,12,0.666667,0.421053,0.516129,edas-iasted
5,15,7,11,0.636364,0.466667,0.538462,cmt-conference
6,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
7,16,5,6,0.833333,0.312500,0.454545,cmt-confof
8,20,11,13,0.846154,0.550000,0.666667,confof-ekaw
9,12,10,11,0.909091,0.833333,0.869565,cmt-sigkdd


In [306]:
candidates = preds_candidates_list[7]
candidates[candidates.string_context_similarity > 0.1]

,source,source_label_x,target,target_label_x,target_label_y,source_label_y,string_similarity,context_similarity,label_embeddings_similarity,label_embeddings_wd_similarity,string_context_similarity,label_emb_context_similarity,label_emb_wd_context_similarity,label_emb_wd_context_all_similarity
0,http://cmt#Administrator,Administrator,http://confOf#Administrator,Administrator,Administrator,Administrator,1.000000,0.731112,1.000000,1.000000,0.731112,0.731112,0.731112,0.731112
1,http://cmt#Author,Author,http://confOf#Author,Author,Author,Author,1.000000,0.799238,1.000000,1.000000,0.799238,0.799238,0.799238,0.799238
2,http://cmt#Conference,Conference,http://confOf#Conference,Conference,Conference,Conference,1.000000,0.737961,1.000000,1.000000,0.737961,0.737961,0.737961,0.737961
3,http://cmt#Paper,Paper,http://confOf#Paper,Paper,Paper,Paper,1.000000,0.527445,1.000000,1.000000,0.527445,0.527445,0.527445,0.527445
4,http://cmt#Person,Person,http://confOf#Person,Person,Person,Person,1.000000,0.698164,1.000000,1.000000,0.698164,0.698164,0.698164,0.698164
403,http://cmt#Rejection,Rejection,http://confOf#Reception,Reception,Reception,Rejection,0.135335,0.823354,0.658037,0.839342,0.111429,0.541797,0.691075,0.454753


In [304]:
maponto.load_alignments(all_refs[7], 'refs')

,source,target,relation,confidence
0,http://cmt#ProgramCommitteeChair,http://confOf#Chair_PC,=,1.0
1,http://cmt#writePaper,http://confOf#writes,=,1.0
2,http://cmt#Author,http://confOf#Author,=,1.0
3,http://cmt#ConferenceMember,http://confOf#Member,=,1.0
4,http://cmt#Administrator,http://confOf#Administrator,=,1.0
5,http://cmt#title,http://confOf#hasTitle,=,1.0
6,http://cmt#SubjectArea,http://confOf#Topic,=,1.0
7,http://cmt#PaperFullVersion,http://confOf#Paper,=,1.0
8,http://cmt#hasBeenAssigned,http://confOf#reviewes,=,1.0
9,http://cmt#hasAuthor,http://confOf#writtenBy,=,1.0


In [17]:
mapneighbor.get_syn_lists('person surname', 5)

[['person', 'individual', 'someone', 'somebody', 'mortal'],
 ['surname', 'family name', 'cognomen', 'last name']]

In [18]:
conf_folder = "../data/conference/with_references"
source_url = conf_folder + "/confof-edas/source.rdf"
target_url = conf_folder + '/confof-edas/target.rdf'
refs_url = conf_folder + '/confof-edas/reference.rdf'

subf = 'confof-edas'

source_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(source_url)
maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
#len(source_graph)

target_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(target_url)
maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
print("len(source_graph)={}, len(target_graph)={}".format(len(source_graph), len(target_graph)))
        
# Concept labels
slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
print("#slabel={}, #tlabel={}".format(slabel_clnd_uris.shape, tlabel_clnd_uris.shape))

sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
print("#sobject={}, #tobject={}".format(sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))

sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
print("#sdatatype={}, #tdatatype={}".format(sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))

../data/conference/with_references/confof-edas/source.rdf
../data/conference/with_references/confof-edas/target.rdf
len(source_graph)=539, len(target_graph)=1720
#slabel=(38, 3), #tlabel=(103, 3)
#sobject=(15, 2), #tobject=(48, 2)
#sdatatype=(32, 2), #tdatatype=(21, 2)


In [8]:
# compute property mathcing based on jaccard similarity of synonym phrases
def compute_property_synListJac_matchings(sproperty_uris, tproperty_uris, threshold, subf):
    """
        input: sproperty_uris: a DataFrame containing source properties ['label', 'uri']
               tproperty_uris: a DataFrame containing target properties ['label', 'uri']
               subf: a string representing the test case
        output: a DataFrame containing candidate property matchings with
                ['source', 'target', 'test_case', 'sLabel', 'tLabel', 
                 'sDomainRangeLabel', 'tDomainRangeLabel',
                 'Jaccard_similarity', 'threshold']
    """
    
    property_matchings = []
    
    for i, srow in tqdm(sproperty_uris.iterrows()):
        # get the uri
        suri = srow['uri']
        sDomainRangeLabel = srow['label']

        # extract the raw label
        slabel = mapneighbor.extract_label_from_uri(suri)

        # clean the label by separating individual words using spaces
        sClndLabel = " ".join(maponto.clean_document_norm(slabel))
        # clean the label by separating individual words using spaces
        sClndDomainRangeLabel = " ".join(maponto.remove_duplicates(maponto.clean_document_keepStops(sDomainRangeLabel)))
        # clean the label by separating individual words using spaces
        sClndDomainRangeLabelNoStop = " ".join(maponto.remove_duplicates(maponto.clean_document_lower(sDomainRangeLabel)))

        #print(sClndLabel)
        #print(sClndDomainRangeLabel)
        #print(mapneighbor.get_syn_phrases(sClndDomainRangeLabelNoStop, 5))
        #print(sClndDomainRangeLabelNoStop)

        sSynLists = mapneighbor.get_syn_lists(sClndDomainRangeLabelNoStop, 5)
        
        for i, trow in tproperty_uris.iterrows():
            turi = trow['uri']
            tDomainRangeLabel = trow['label']


            # extract the raw label
            tlabel = mapneighbor.extract_label_from_uri(turi)


            # clean the label by separating individual words using spaces
            tClndLabel = " ".join(maponto.clean_document_norm(tlabel))
            # clean the label by separating individual words using spaces
            tClndDomainRangeLabel = " ".join(maponto.remove_duplicates(maponto.clean_document_keepStops(tDomainRangeLabel)))
            # clean the label by separating individual words using spaces
            tClndDomainRangeLabelNoStop = " ".join(maponto.remove_duplicates(maponto.clean_document_lower(tDomainRangeLabel)))

            #print(tClndLabel)
            #print(tClndDomainRangeLabel)
            #print(mapneighbor.get_syn_phrases(tClndDomainRangeLabelNoStop, 5))
            #print(tClndDomainRangeLabelNoStop)

            tSynLists = mapneighbor.get_syn_lists(tClndDomainRangeLabelNoStop, 5)
            
            size_union = len(sSynLists) + len(tSynLists)
            size_intersection = 0
            total_size = size_union
            #print('size_union', size_union)
            
            #print('sSynLists', sSynLists)
            #print('tSynLists', tSynLists)
            tmapped = []
            for sList in sSynLists:
                #print('sList', sList)
                
                for i, tList in enumerate(tSynLists):
                    if (i not in tmapped) & (len(set(sList).intersection(set(tList))) > 0):
                        #print('sList', sList)
                        #print('tList', tList)
                        size_intersection = size_intersection + 1
                        size_union = size_union - 1
                        tmapped.append(i)
                        break
                        
            #print('size_intersection', size_intersection)
            #print('final size_union', size_union)
            
            jacSim = size_intersection / size_union
            
            if (jacSim >= threshold):
                #print('sSynLists', sSynLists)
                #print('tSynLists', tSynLists)
                #print('Jaccard similarity', jacSim)
                #print('size_intersction', size_intersection)
                #print('size_union', size_union)
                #print('total_size', total_size)
                amap = {'source':suri, 'target':turi, 
                        'test_case':subf,
                        'sLabel':slabel,
                        'tLabel':tlabel,
                        'sDomainRangeLabel':sDomainRangeLabel,
                        'tDomainRangeLabel':tDomainRangeLabel,
                        'Jaccard_similarity': jacSim,
                        'threshold':threshold}
                        
                property_matchings.append(amap)
                     
    aligns = pd.DataFrame(property_matchings)
    
    if len(aligns) > 0:
        aligns = aligns.sort_values('Jaccard_similarity', ascending=False)
    
        idx = []
        sources = []
        targets = []
        for j, align in aligns.iterrows():
            if (align['source'] not in sources) & (align['target'] not in targets):
                sources.append(align['source'])
                targets.append(align['target'])
                idx.append(j)

        return aligns.loc[idx].reset_index(drop=True)
    else:
        return pd.DataFrame(columns=['source', 'target'])

In [62]:
mapneighbor.get_syn_lists('person sur name', 5)

[['person', 'individual', 'someone', 'somebody', 'mortal'],
 ['sur', 'tyre'],
 ['name', 'gens', 'figure', 'public figure', 'epithet']]

In [63]:
%%time
aligns = compute_property_synListJac_matchings(sdatatype_domain_uris, tdatatype_domain_uris, 0.5, subf)
aligns

32it [00:06,  4.85it/s]

CPU times: user 6.22 s, sys: 393 ms, total: 6.62 s
Wall time: 6.61 s


,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://confOf#hasEmail,http://edas#hasEmail,confof-edas,hasEmail,hasEmail,Person hasEmail,Person hasEmail,1.0,0.5
1,http://confOf#hasFirstName,http://edas#hasFirstName,confof-edas,hasFirstName,hasFirstName,Person hasFirstName,Person hasFirstName,1.0,0.5


In [64]:
aligns_evaluation = maponto.evaluate_noprint(aligns, refs_url)
aligns_evaluation['test_case']= subf
pd.DataFrame([aligns_evaluation])                

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,19,1,2,0.5,0.052632,0.095238,confof-edas


In [65]:
aligns = compute_property_synListJac_matchings(sobject_domain_range_uris, tobject_domain_range_uris, 0.5, subf)
aligns

15it [00:06,  2.37it/s]


,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://confOf#writtenBy,http://edas#isWrittenBy,confof-edas,writtenBy,isWrittenBy,Contribution writtenBy Author,Paper isWrittenBy Author,0.5,0.5


In [66]:
aligns_evaluation = maponto.evaluate_noprint(aligns, refs_url)
aligns_evaluation['test_case']= subf
pd.DataFrame([aligns_evaluation])    

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,19,1,1,1.0,0.052632,0.1,confof-edas


In [67]:
%%time

import time
import os
all_refined_class_aligns_list_05 = []
all_class_evaluations_list_05 = []

all_refined_object_aligns_list_05 = []
all_object_evaluations_list_05 = []

all_refined_datatype_aligns_list_05 = []
all_datatype_evaluations_list_05 = []

all_refined_class_property_aligns_list_05 = []
all_class_property_evaluations_list_05 = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.25]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list_05.append(refined_class_aligns)
        all_class_evaluations_list_05.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synListJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.5, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list_05.append(refined_object_aligns)
                all_object_evaluations_list_05.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list_05.append(None)
                all_object_evaluations_list_05.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list_05.append(None)
            all_object_evaluations_list_05.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synListJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.5, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list_05.append(refined_datatype_aligns)
                all_datatype_evaluations_list_05.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list_05.append(None)
                all_datatype_evaluations_list_05.append(None)
        else:
            all_refined_datatype_aligns_list_05.append(None)
            all_datatype_evaluations_list_05.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list_05.append(aligns_class_property)
        all_class_property_evaluations_list_05.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 64.24it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [00:16,  3.19it/s]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 49min 50s, sys: 1min 8s, total: 50min 58s
Wall time: 12min 18s


In [68]:
len(all_refined_class_property_aligns_list_05)

21

In [69]:
pd.concat(all_class_property_evaluations_list_05)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,14,0.857143,0.800000,0.827586,iasted-sigkdd
0,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd
0,23,12,21,0.571429,0.521739,0.545455,edas-ekaw
0,15,8,11,0.727273,0.533333,0.615385,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,12,0.416667,0.333333,0.370370,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,5,7,0.714286,0.312500,0.434783,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,9,10,0.900000,0.750000,0.818182,cmt-sigkdd


In [70]:
pd.concat(all_class_property_evaluations_list_05).mean()

/tmp/ipykernel_99446/4220453350.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list_05).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.761905
numOfPredicted             10.380952
precision                   0.781917
recall                      0.566259
f1                          0.642241
dtype: float64

In [42]:
pd.concat(all_class_evaluations_list_05)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.25,0.087551,one-to-one
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd,string_context_similarity,0.25,0.109206,one-to-one
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw,string_context_similarity,0.25,0.235250,one-to-one
0,15,8,9,0.888889,0.533333,0.666667,conference-sigkdd,string_context_similarity,0.25,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.25,0.081515,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.25,0.042640,one-to-one
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted,string_context_similarity,0.25,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.25,0.111429,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.25,0.037791,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.25,0.038805,one-to-one


In [43]:
pd.concat(all_class_evaluations_list_05).mean()

/tmp/ipykernel_80191/173897161.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list_05).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.857143
numOfPredicted              7.809524
precision                   0.889611
recall                      0.505198
f1                          0.631327
similarity_threshold        0.250000
similarity_cutoff           0.083662
dtype: float64

In [44]:
pd.concat(all_object_evaluations_list_05)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,0,1,0.000000,0.000000,0.000000,iasted-sigkdd
0,23,3,10,0.300000,0.130435,0.181818,edas-ekaw
0,15,0,4,0.000000,0.000000,0.000000,cmt-conference
0,12,1,1,1.000000,0.083333,0.153846,cmt-sigkdd
0,19,1,1,1.000000,0.052632,0.100000,confof-edas
0,11,3,4,0.750000,0.272727,0.400000,cmt-ekaw
0,9,0,2,0.000000,0.000000,0.000000,confof-iasted
0,15,0,1,0.000000,0.000000,0.000000,edas-sigkdd
0,17,0,5,0.000000,0.000000,0.000000,conference-edas
0,13,1,9,0.111111,0.076923,0.090909,cmt-edas


In [45]:
pd.concat(all_object_evaluations_list_05).mean()

/tmp/ipykernel_80191/2667318125.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list_05).mean()


numOfRefs                  15.818182
numOfCorrectlyPredicted     0.818182
numOfPredicted              3.727273
precision                   0.287374
recall                      0.056005
f1                          0.084234
dtype: float64

In [46]:
pd.concat(all_datatype_evaluations_list_05)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,2,0.500000,0.142857,0.222222,confof-sigkdd
0,15,1,4,0.250000,0.066667,0.105263,conference-sigkdd
0,19,0,1,0.000000,0.000000,0.000000,edas-iasted
0,15,1,4,0.250000,0.066667,0.105263,cmt-conference
0,16,2,3,0.666667,0.125000,0.210526,cmt-confof
0,12,1,3,0.333333,0.083333,0.133333,cmt-sigkdd
0,19,2,4,0.500000,0.105263,0.173913,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,3,5,0.600000,0.200000,0.300000,edas-sigkdd
0,17,2,7,0.285714,0.117647,0.166667,conference-edas


In [47]:
pd.concat(all_datatype_evaluations_list_05).mean()

/tmp/ipykernel_80191/2399751814.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list_05).mean()


numOfRefs                  14.818182
numOfCorrectlyPredicted     1.454545
numOfPredicted              3.636364
precision                   0.398701
recall                      0.100676
f1                          0.159138
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases+ Jaccard on Sets 0.75

In [71]:
%%time

import time
import os
all_refined_class_aligns_list_075 = []
all_class_evaluations_list_075 = []

all_refined_object_aligns_list_075 = []
all_object_evaluations_list_075 = []

all_refined_datatype_aligns_list_075 = []
all_datatype_evaluations_list_075 = []

all_refined_class_property_aligns_list_075 = []
all_class_property_evaluations_list_075 = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.25]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list_075.append(refined_class_aligns)
        all_class_evaluations_list_075.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synListJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.75, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list_075.append(refined_object_aligns)
                all_object_evaluations_list_075.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list_075.append(None)
                all_object_evaluations_list_075.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list_075.append(None)
            all_object_evaluations_list_075.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synListJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.75, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list_075.append(refined_datatype_aligns)
                all_datatype_evaluations_list_075.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list_075.append(None)
                all_datatype_evaluations_list_075.append(None)
        else:
            all_refined_datatype_aligns_list_075.append(None)
            all_datatype_evaluations_list_075.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list_075.append(aligns_class_property)
        all_class_property_evaluations_list_075.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 60.68it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [00:17,  3.02it/s]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 50min 56s, sys: 1min 14s, total: 52min 11s
Wall time: 12min 30s


In [72]:
len(all_refined_class_property_aligns_list_075)

21

In [73]:
pd.concat(all_class_property_evaluations_list_075)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,8,0.625000,0.333333,0.434783,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,5,6,0.833333,0.312500,0.454545,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,9,10,0.900000,0.750000,0.818182,cmt-sigkdd


In [74]:
pd.concat(all_class_property_evaluations_list_075).mean()

/tmp/ipykernel_99446/1225595666.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list_075).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.333333
numOfPredicted              8.714286
precision                   0.862287
recall                      0.537279
f1                          0.649329
dtype: float64

In [52]:
pd.concat(all_class_evaluations_list_075)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.25,0.087551,one-to-one
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd,string_context_similarity,0.25,0.109206,one-to-one
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw,string_context_similarity,0.25,0.235250,one-to-one
0,15,8,9,0.888889,0.533333,0.666667,conference-sigkdd,string_context_similarity,0.25,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.25,0.081515,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.25,0.042640,one-to-one
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted,string_context_similarity,0.25,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.25,0.111429,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.25,0.037791,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.25,0.038805,one-to-one


In [53]:
pd.concat(all_class_evaluations_list_075).mean()

/tmp/ipykernel_80191/2438927775.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list_075).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.857143
numOfPredicted              7.809524
precision                   0.889611
recall                      0.505198
f1                          0.631327
similarity_threshold        0.250000
similarity_cutoff           0.083662
dtype: float64

In [54]:
pd.concat(all_object_evaluations_list_075)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,23,0,1,0.0,0.000000,0.000000,edas-ekaw
0,15,0,1,0.0,0.000000,0.000000,cmt-conference
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd
0,11,1,1,1.0,0.090909,0.166667,cmt-ekaw
0,9,0,1,0.0,0.000000,0.000000,confof-iasted
0,13,2,2,1.0,0.153846,0.266667,cmt-edas
0,25,0,1,0.0,0.000000,0.000000,conference-ekaw


In [55]:
pd.concat(all_object_evaluations_list_075).mean()

/tmp/ipykernel_80191/1601419568.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list_075).mean()


numOfRefs                  15.428571
numOfCorrectlyPredicted     0.571429
numOfPredicted              1.142857
precision                   0.428571
recall                      0.046870
f1                          0.083883
dtype: float64

In [56]:
pd.concat(all_datatype_evaluations_list_075)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,2,0.500000,0.142857,0.222222,confof-sigkdd
0,15,1,3,0.333333,0.066667,0.111111,conference-sigkdd
0,15,1,4,0.250000,0.066667,0.105263,cmt-conference
0,16,1,2,0.500000,0.062500,0.111111,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,3,5,0.600000,0.200000,0.300000,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,3,0.000000,0.000000,0.000000,cmt-edas


In [57]:
pd.concat(all_datatype_evaluations_list_075).mean()

/tmp/ipykernel_80191/1508283105.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list_075).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.500000
numOfPredicted              3.100000
precision                   0.485000
recall                      0.104493
f1                          0.169819
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases+ Jaccard on Sets 0.80

In [75]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.25]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synListJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.80, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synListJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.80, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 63.84it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [00:17,  3.02it/s]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 50min 31s, sys: 1min 8s, total: 51min 39s
Wall time: 12min 26s


In [76]:
len(all_refined_class_property_aligns_list)

21

In [77]:
pd.concat(all_class_property_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,8,0.625000,0.333333,0.434783,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,5,6,0.833333,0.312500,0.454545,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,9,10,0.900000,0.750000,0.818182,cmt-sigkdd


In [78]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_99446/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.285714
numOfPredicted              8.619048
precision                   0.864642
recall                      0.532950
f1                          0.646055
dtype: float64

In [32]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.25,0.087551,one-to-one
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd,string_context_similarity,0.25,0.109206,one-to-one
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw,string_context_similarity,0.25,0.235250,one-to-one
0,15,8,9,0.888889,0.533333,0.666667,conference-sigkdd,string_context_similarity,0.25,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.25,0.081515,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.25,0.042640,one-to-one
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted,string_context_similarity,0.25,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.25,0.111429,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.25,0.037791,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.25,0.038805,one-to-one


In [33]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_80191/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.857143
numOfPredicted              7.809524
precision                   0.889611
recall                      0.505198
f1                          0.631327
similarity_threshold        0.250000
similarity_cutoff           0.083662
dtype: float64

In [34]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,0,1,0.0,0.000000,0.000000,cmt-conference
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd
0,13,2,2,1.0,0.153846,0.266667,cmt-edas
0,25,0,1,0.0,0.000000,0.000000,conference-ekaw


In [35]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_80191/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  16.250000
numOfCorrectlyPredicted     0.750000
numOfPredicted              1.250000
precision                   0.500000
recall                      0.059295
f1                          0.105128
dtype: float64

In [36]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,1,1.000000,0.142857,0.250000,confof-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,conference-sigkdd
0,15,1,3,0.333333,0.066667,0.111111,cmt-conference
0,16,1,1,1.000000,0.062500,0.117647,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,2,3,0.666667,0.133333,0.222222,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,3,0.000000,0.000000,0.000000,cmt-edas


In [37]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_80191/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.400000
numOfPredicted              2.500000
precision                   0.616667
recall                      0.097827
f1                          0.166711
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jaccard on Sets 0.85

In [79]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.25]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synListJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.85, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synListJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.85, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 66.58it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [00:17,  3.13it/s]

21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 51min 12s, sys: 1min 7s, total: 52min 20s
Wall time: 12min 29s


In [80]:
len(all_refined_class_property_aligns_list)

21

In [81]:
pd.concat(all_class_property_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,8,0.625000,0.333333,0.434783,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,5,6,0.833333,0.312500,0.454545,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,9,10,0.900000,0.750000,0.818182,cmt-sigkdd


In [82]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_99446/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.285714
numOfPredicted              8.619048
precision                   0.864642
recall                      0.532950
f1                          0.646055
dtype: float64

In [237]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.62,0.274400,one-to-one
0,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd,string_context_similarity,0.62,0.109206,one-to-one
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw,string_context_similarity,0.62,0.619406,one-to-one
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd,string_context_similarity,0.62,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.62,0.265139,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.62,0.559763,one-to-one
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted,string_context_similarity,0.62,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.62,0.527445,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.62,0.316072,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.62,0.571298,one-to-one


In [238]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_60200/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              8.047619
precision                   0.849485
recall                      0.503128
f1                          0.615739
similarity_threshold        0.620000
similarity_cutoff           0.357616
dtype: float64

In [239]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,0,1,0.0,0.000000,0.000000,cmt-conference
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd
0,13,2,2,1.0,0.153846,0.266667,cmt-edas
0,25,0,1,0.0,0.000000,0.000000,conference-ekaw


In [240]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_60200/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  16.250000
numOfCorrectlyPredicted     0.750000
numOfPredicted              1.250000
precision                   0.500000
recall                      0.059295
f1                          0.105128
dtype: float64

In [241]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,1,1.000000,0.142857,0.250000,confof-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,conference-sigkdd
0,15,1,3,0.333333,0.066667,0.111111,cmt-conference
0,16,1,1,1.000000,0.062500,0.117647,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,2,3,0.666667,0.133333,0.222222,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,3,0.000000,0.000000,0.000000,cmt-edas


In [242]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_60200/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.400000
numOfPredicted              2.500000
precision                   0.616667
recall                      0.097827
f1                          0.166711
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jaccard on Sets 0.9

In [83]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.62]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synListJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.90, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synListJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.90, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 47.42it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [00:17,  3.03it/s]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 51min 5s, sys: 1min 8s, total: 52min 13s
Wall time: 12min 21s


In [84]:
len(all_refined_class_property_aligns_list)

21

In [85]:
pd.concat(all_class_property_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,11,12,0.916667,0.733333,0.814815,iasted-sigkdd
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd
0,23,9,11,0.818182,0.391304,0.529412,edas-ekaw
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd
0,19,7,8,0.875000,0.368421,0.518519,edas-iasted
0,15,5,7,0.714286,0.333333,0.454545,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,5,5,1.000000,0.312500,0.476190,cmt-confof
0,20,8,9,0.888889,0.400000,0.551724,confof-ekaw
0,12,8,9,0.888889,0.666667,0.761905,cmt-sigkdd


In [86]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_99446/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              7.857143
precision                   0.884459
recall                      0.500487
f1                          0.622863
dtype: float64

In [247]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.62,0.274400,one-to-one
0,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd,string_context_similarity,0.62,0.109206,one-to-one
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw,string_context_similarity,0.62,0.619406,one-to-one
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd,string_context_similarity,0.62,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.62,0.265139,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.62,0.559763,one-to-one
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted,string_context_similarity,0.62,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.62,0.527445,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.62,0.316072,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.62,0.571298,one-to-one


In [248]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_60200/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              8.047619
precision                   0.849485
recall                      0.503128
f1                          0.615739
similarity_threshold        0.620000
similarity_cutoff           0.357616
dtype: float64

In [249]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd


In [250]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_60200/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  12.000000
numOfCorrectlyPredicted     1.000000
numOfPredicted              1.000000
precision                   1.000000
recall                      0.083333
f1                          0.153846
dtype: float64

In [251]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,1,1.000000,0.142857,0.250000,confof-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,conference-sigkdd
0,15,1,3,0.333333,0.066667,0.111111,cmt-conference
0,16,1,1,1.000000,0.062500,0.117647,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,1,2,0.500000,0.066667,0.117647,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,3,0.000000,0.000000,0.000000,cmt-edas


In [252]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_60200/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.300000
numOfPredicted              2.400000
precision                   0.600000
recall                      0.091160
f1                          0.156254
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jaccard on Sets 0.95

In [87]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.62]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synListJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.95, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synListJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.95, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 66.56it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [00:17,  3.05it/s]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 52min 28s, sys: 1min 9s, total: 53min 38s
Wall time: 12min 41s


In [88]:
len(all_refined_class_property_aligns_list)

21

In [89]:
pd.concat(all_class_property_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,11,12,0.916667,0.733333,0.814815,iasted-sigkdd
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd
0,23,9,11,0.818182,0.391304,0.529412,edas-ekaw
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd
0,19,7,8,0.875000,0.368421,0.518519,edas-iasted
0,15,5,7,0.714286,0.333333,0.454545,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,5,5,1.000000,0.312500,0.476190,cmt-confof
0,20,8,9,0.888889,0.400000,0.551724,confof-ekaw
0,12,8,9,0.888889,0.666667,0.761905,cmt-sigkdd


In [90]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_99446/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              7.857143
precision                   0.884459
recall                      0.500487
f1                          0.622863
dtype: float64

In [257]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.62,0.274400,one-to-one
0,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd,string_context_similarity,0.62,0.109206,one-to-one
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw,string_context_similarity,0.62,0.619406,one-to-one
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd,string_context_similarity,0.62,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.62,0.265139,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.62,0.559763,one-to-one
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted,string_context_similarity,0.62,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.62,0.527445,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.62,0.316072,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.62,0.571298,one-to-one


In [258]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_60200/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              8.047619
precision                   0.849485
recall                      0.503128
f1                          0.615739
similarity_threshold        0.620000
similarity_cutoff           0.357616
dtype: float64

In [259]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd


In [260]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_60200/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  12.000000
numOfCorrectlyPredicted     1.000000
numOfPredicted              1.000000
precision                   1.000000
recall                      0.083333
f1                          0.153846
dtype: float64

In [261]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,1,1.000000,0.142857,0.250000,confof-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,conference-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,cmt-conference
0,16,1,1,1.000000,0.062500,0.117647,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,1,2,0.500000,0.066667,0.117647,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,2,0.000000,0.000000,0.000000,cmt-edas


In [262]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_60200/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.300000
numOfPredicted              2.200000
precision                   0.616667
recall                      0.091160
f1                          0.156907
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jac 0.5

In [7]:
conf_folder = "../data/conference/with_references"
source_url = conf_folder + "/confof-edas/source.rdf"
target_url = conf_folder + '/confof-edas/target.rdf'
refs_url = conf_folder + '/confof-edas/reference.rdf'

subf = 'confof-edas'

source_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(source_url)
maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
#len(source_graph)

target_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(target_url)
maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
print("len(source_graph)={}, len(target_graph)={}".format(len(source_graph), len(target_graph)))
        
# Concept labels
slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
print("#slabel={}, #tlabel={}".format(slabel_clnd_uris.shape, tlabel_clnd_uris.shape))

sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
print("#sobject={}, #tobject={}".format(sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))

sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
print("#sdatatype={}, #tdatatype={}".format(sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))

../data/conference/with_references/confof-edas/source.rdf
../data/conference/with_references/confof-edas/target.rdf
len(source_graph)=539, len(target_graph)=1720
#slabel=(38, 3), #tlabel=(103, 3)
#sobject=(15, 2), #tobject=(48, 2)
#sdatatype=(32, 2), #tdatatype=(21, 2)


In [9]:
# compute property mathcing based on jaccard similarity of synonym phrases
def compute_property_synJac_matchings(sproperty_uris, tproperty_uris, threshold, subf):
    """
        input: sproperty_uris: a DataFrame containing source properties ['label', 'uri']
               tproperty_uris: a DataFrame containing target properties ['label', 'uri']
               subf: a string representing the test case
        output: a DataFrame containing candidate property matchings with
                ['source', 'target', 'test_case', 'sLabel', 'tLabel', 
                 'sDomainRangeLabel', 'tDomainRangeLabel',
                 'Jaccard_similarity', 'threshold']
    """
    
    property_matchings = []
    
    for i, srow in tqdm(sproperty_uris.iterrows()):
        # get the uri
        suri = srow['uri']
        sDomainRangeLabel = srow['label']

        # extract the raw label
        slabel = mapneighbor.extract_label_from_uri(suri)

        # clean the label by separating individual words using spaces
        sClndLabel = " ".join(maponto.clean_document_norm(slabel))
        # clean the label by separating individual words using spaces
        sClndDomainRangeLabel = " ".join(maponto.remove_duplicates(maponto.clean_document_keepStops(sDomainRangeLabel)))
        # clean the label by separating individual words using spaces
        sClndDomainRangeLabelNoStop = " ".join(maponto.remove_duplicates(maponto.clean_document_lower(sDomainRangeLabel)))

        #print(sClndLabel)
        #print(sClndDomainRangeLabel)
        #print(mapneighbor.get_syn_phrases(sClndDomainRangeLabelNoStop, 5))
        #print(sClndDomainRangeLabelNoStop)

        sSynLabels = mapneighbor.get_syn_phrases(sClndDomainRangeLabelNoStop, 5)
        
        for i, trow in tproperty_uris.iterrows():
            turi = trow['uri']
            tDomainRangeLabel = trow['label']


            # extract the raw label
            tlabel = mapneighbor.extract_label_from_uri(turi)


            # clean the label by separating individual words using spaces
            tClndLabel = " ".join(maponto.clean_document_norm(tlabel))
            # clean the label by separating individual words using spaces
            tClndDomainRangeLabel = " ".join(maponto.remove_duplicates(maponto.clean_document_keepStops(tDomainRangeLabel)))
            # clean the label by separating individual words using spaces
            tClndDomainRangeLabelNoStop = " ".join(maponto.remove_duplicates(maponto.clean_document_lower(tDomainRangeLabel)))

            #print(tClndLabel)
            #print(tClndDomainRangeLabel)
            #print(mapneighbor.get_syn_phrases(tClndDomainRangeLabelNoStop, 5))
            #print(tClndDomainRangeLabelNoStop)

            tSynLabels = mapneighbor.get_syn_phrases(tClndDomainRangeLabelNoStop, 5)
            
            mapped = False
    
            for sSynLabel in sSynLabels:
                for tSynLabel in tSynLabels:
                    jacSim = mapneighbor.jaccard_similarity(sSynLabel.split(), 
                                                            tSynLabel.split())
                    #print(jacSim)
                    if (jacSim >= threshold):
                        amap = {'source':suri, 'target':turi, 
                                'test_case':subf,
                                'sLabel':slabel,
                                'tLabel':tlabel,
                                'sDomainRangeLabel':sDomainRangeLabel,
                                'tDomainRangeLabel':tDomainRangeLabel,
                                'Jaccard_similarity': jacSim,
                                'threshold':threshold}
                        
                        mapped = True
                        property_matchings.append(amap)
                        break
                if mapped:
                    break
    
    aligns = pd.DataFrame(property_matchings)
    
    if len(aligns) > 0:
        aligns = aligns.sort_values('Jaccard_similarity', ascending=False)
    
        idx = []
        sources = []
        targets = []
        for j, align in aligns.iterrows():
            if (align['source'] not in sources) & (align['target'] not in targets):
                sources.append(align['source'])
                targets.append(align['target'])
                idx.append(j)

        return aligns.loc[idx].reset_index(drop=True)
    else:
        return pd.DataFrame(columns=['source', 'target'])

In [10]:
%%time
aligns = compute_property_synJac_matchings(sdatatype_domain_uris, tdatatype_domain_uris, 0.5, subf)
aligns

32it [01:01,  1.91s/it]

CPU times: user 1min, sys: 419 ms, total: 1min 1s
Wall time: 1min 1s


,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://confOf#hasEmail,http://edas#hasEmail,confof-edas,hasEmail,hasEmail,Person hasEmail,Person hasEmail,1.0,0.5
1,http://confOf#hasFirstName,http://edas#hasFirstName,confof-edas,hasFirstName,hasFirstName,Person hasFirstName,Person hasFirstName,1.0,0.5
2,http://confOf#hasPostalCode,http://edas#hasPostalCode,confof-edas,hasPostalCode,hasPostalCode,Person hasPostalCode,ContactInformation hasPostalCode,0.5,0.5
3,http://confOf#hasSurname,http://edas#hasLastName,confof-edas,hasSurname,hasLastName,Person hasSurname,Person hasLastName,0.5,0.5


In [11]:
aligns_evaluation = maponto.evaluate_noprint(aligns, refs_url)
aligns_evaluation['test_case']= subf
pd.DataFrame([aligns_evaluation])                

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,19,2,4,0.5,0.105263,0.173913,confof-edas


In [12]:
aligns = compute_property_synJac_matchings(sobject_domain_range_uris, tobject_domain_range_uris, 0.5, subf)
aligns

15it [00:13,  1.07it/s]


,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://confOf#writtenBy,http://edas#isWrittenBy,confof-edas,writtenBy,isWrittenBy,Contribution writtenBy Author,Paper isWrittenBy Author,0.5,0.5


In [13]:
aligns_evaluation = maponto.evaluate_noprint(aligns, refs_url)
aligns_evaluation['test_case']= subf
pd.DataFrame([aligns_evaluation])    

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,19,1,1,1.0,0.052632,0.1,confof-edas


In [14]:
%%time

import time
import os
all_refined_class_aligns_list_05 = []
all_class_evaluations_list_05 = []

all_refined_object_aligns_list_05 = []
all_object_evaluations_list_05 = []

all_refined_datatype_aligns_list_05 = []
all_datatype_evaluations_list_05 = []

all_refined_class_property_aligns_list_05 = []
all_class_property_evaluations_list_05 = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.25]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list_05.append(refined_class_aligns)
        all_class_evaluations_list_05.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.5, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list_05.append(refined_object_aligns)
                all_object_evaluations_list_05.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list_05.append(None)
                all_object_evaluations_list_05.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list_05.append(None)
            all_object_evaluations_list_05.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.5, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list_05.append(refined_datatype_aligns)
                all_datatype_evaluations_list_05.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list_05.append(None)
                all_datatype_evaluations_list_05.append(None)
        else:
            all_refined_datatype_aligns_list_05.append(None)
            all_datatype_evaluations_list_05.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list_05.append(aligns_class_property)
        all_class_property_evaluations_list_05.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 88.86it/s]


6. #sobject=(51, 2), #tobject=(54, 2)


14it [04:22, 18.77s/it]


KeyboardInterrupt: 

In [39]:
len(all_refined_class_property_aligns_list_05)

21

In [40]:
pd.concat(all_class_property_evaluations_list_05)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,14,0.857143,0.800000,0.827586,iasted-sigkdd
0,7,5,6,0.833333,0.714286,0.769231,confof-sigkdd
0,23,13,23,0.565217,0.565217,0.565217,edas-ekaw
0,15,9,13,0.692308,0.600000,0.642857,conference-sigkdd
0,19,8,10,0.800000,0.421053,0.551724,edas-iasted
0,15,5,14,0.357143,0.333333,0.344828,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,6,8,0.750000,0.375000,0.500000,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,10,12,0.833333,0.833333,0.833333,cmt-sigkdd


In [41]:
pd.concat(all_class_property_evaluations_list_05).mean()

/tmp/ipykernel_80191/4220453350.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list_05).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     8.047619
numOfPredicted             11.666667
precision                   0.744857
recall                      0.587269
f1                          0.639874
dtype: float64

In [42]:
pd.concat(all_class_evaluations_list_05)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.25,0.087551,one-to-one
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd,string_context_similarity,0.25,0.109206,one-to-one
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw,string_context_similarity,0.25,0.235250,one-to-one
0,15,8,9,0.888889,0.533333,0.666667,conference-sigkdd,string_context_similarity,0.25,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.25,0.081515,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.25,0.042640,one-to-one
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted,string_context_similarity,0.25,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.25,0.111429,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.25,0.037791,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.25,0.038805,one-to-one


In [43]:
pd.concat(all_class_evaluations_list_05).mean()

/tmp/ipykernel_80191/173897161.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list_05).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.857143
numOfPredicted              7.809524
precision                   0.889611
recall                      0.505198
f1                          0.631327
similarity_threshold        0.250000
similarity_cutoff           0.083662
dtype: float64

In [44]:
pd.concat(all_object_evaluations_list_05)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,0,1,0.000000,0.000000,0.000000,iasted-sigkdd
0,23,3,10,0.300000,0.130435,0.181818,edas-ekaw
0,15,0,4,0.000000,0.000000,0.000000,cmt-conference
0,12,1,1,1.000000,0.083333,0.153846,cmt-sigkdd
0,19,1,1,1.000000,0.052632,0.100000,confof-edas
0,11,3,4,0.750000,0.272727,0.400000,cmt-ekaw
0,9,0,2,0.000000,0.000000,0.000000,confof-iasted
0,15,0,1,0.000000,0.000000,0.000000,edas-sigkdd
0,17,0,5,0.000000,0.000000,0.000000,conference-edas
0,13,1,9,0.111111,0.076923,0.090909,cmt-edas


In [45]:
pd.concat(all_object_evaluations_list_05).mean()

/tmp/ipykernel_80191/2667318125.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list_05).mean()


numOfRefs                  15.818182
numOfCorrectlyPredicted     0.818182
numOfPredicted              3.727273
precision                   0.287374
recall                      0.056005
f1                          0.084234
dtype: float64

In [46]:
pd.concat(all_datatype_evaluations_list_05)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,2,0.500000,0.142857,0.222222,confof-sigkdd
0,15,1,4,0.250000,0.066667,0.105263,conference-sigkdd
0,19,0,1,0.000000,0.000000,0.000000,edas-iasted
0,15,1,4,0.250000,0.066667,0.105263,cmt-conference
0,16,2,3,0.666667,0.125000,0.210526,cmt-confof
0,12,1,3,0.333333,0.083333,0.133333,cmt-sigkdd
0,19,2,4,0.500000,0.105263,0.173913,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,3,5,0.600000,0.200000,0.300000,edas-sigkdd
0,17,2,7,0.285714,0.117647,0.166667,conference-edas


In [47]:
pd.concat(all_datatype_evaluations_list_05).mean()

/tmp/ipykernel_80191/2399751814.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list_05).mean()


numOfRefs                  14.818182
numOfCorrectlyPredicted     1.454545
numOfPredicted              3.636364
precision                   0.398701
recall                      0.100676
f1                          0.159138
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jac 0.75

In [48]:
%%time

import time
import os
all_refined_class_aligns_list_075 = []
all_class_evaluations_list_075 = []

all_refined_object_aligns_list_075 = []
all_object_evaluations_list_075 = []

all_refined_datatype_aligns_list_075 = []
all_datatype_evaluations_list_075 = []

all_refined_class_property_aligns_list_075 = []
all_class_property_evaluations_list_075 = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.25]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list_075.append(refined_class_aligns)
        all_class_evaluations_list_075.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.75, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list_075.append(refined_object_aligns)
                all_object_evaluations_list_075.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list_075.append(None)
                all_object_evaluations_list_075.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list_075.append(None)
            all_object_evaluations_list_075.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.75, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list_075.append(refined_datatype_aligns)
                all_datatype_evaluations_list_075.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list_075.append(None)
                all_datatype_evaluations_list_075.append(None)
        else:
            all_refined_datatype_aligns_list_075.append(None)
            all_datatype_evaluations_list_075.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list_075.append(aligns_class_property)
        all_class_property_evaluations_list_075.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 66.69it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [11:22, 12.63s/it]

21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 3h 56min 43s, sys: 1min 11s, total: 3h 57min 54s
Wall time: 3h 17min 50s


In [49]:
len(all_refined_class_property_aligns_list_075)

21

In [50]:
pd.concat(all_class_property_evaluations_list_075)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd
0,7,5,6,0.833333,0.714286,0.769231,confof-sigkdd
0,23,10,14,0.714286,0.434783,0.540541,edas-ekaw
0,15,9,12,0.750000,0.600000,0.666667,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,11,0.454545,0.333333,0.384615,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,5,7,0.714286,0.312500,0.434783,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,10,11,0.909091,0.833333,0.869565,cmt-sigkdd


In [51]:
pd.concat(all_class_property_evaluations_list_075).mean()

/tmp/ipykernel_80191/1225595666.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list_075).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.761905
numOfPredicted              9.666667
precision                   0.818118
recall                      0.570580
f1                          0.658916
dtype: float64

In [52]:
pd.concat(all_class_evaluations_list_075)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.25,0.087551,one-to-one
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd,string_context_similarity,0.25,0.109206,one-to-one
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw,string_context_similarity,0.25,0.235250,one-to-one
0,15,8,9,0.888889,0.533333,0.666667,conference-sigkdd,string_context_similarity,0.25,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.25,0.081515,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.25,0.042640,one-to-one
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted,string_context_similarity,0.25,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.25,0.111429,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.25,0.037791,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.25,0.038805,one-to-one


In [53]:
pd.concat(all_class_evaluations_list_075).mean()

/tmp/ipykernel_80191/2438927775.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list_075).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.857143
numOfPredicted              7.809524
precision                   0.889611
recall                      0.505198
f1                          0.631327
similarity_threshold        0.250000
similarity_cutoff           0.083662
dtype: float64

In [54]:
pd.concat(all_object_evaluations_list_075)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,23,0,1,0.0,0.000000,0.000000,edas-ekaw
0,15,0,1,0.0,0.000000,0.000000,cmt-conference
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd
0,11,1,1,1.0,0.090909,0.166667,cmt-ekaw
0,9,0,1,0.0,0.000000,0.000000,confof-iasted
0,13,2,2,1.0,0.153846,0.266667,cmt-edas
0,25,0,1,0.0,0.000000,0.000000,conference-ekaw


In [55]:
pd.concat(all_object_evaluations_list_075).mean()

/tmp/ipykernel_80191/1601419568.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list_075).mean()


numOfRefs                  15.428571
numOfCorrectlyPredicted     0.571429
numOfPredicted              1.142857
precision                   0.428571
recall                      0.046870
f1                          0.083883
dtype: float64

In [56]:
pd.concat(all_datatype_evaluations_list_075)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,2,0.500000,0.142857,0.222222,confof-sigkdd
0,15,1,3,0.333333,0.066667,0.111111,conference-sigkdd
0,15,1,4,0.250000,0.066667,0.105263,cmt-conference
0,16,1,2,0.500000,0.062500,0.111111,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,3,5,0.600000,0.200000,0.300000,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,3,0.000000,0.000000,0.000000,cmt-edas


In [57]:
pd.concat(all_datatype_evaluations_list_075).mean()

/tmp/ipykernel_80191/1508283105.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list_075).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.500000
numOfPredicted              3.100000
precision                   0.485000
recall                      0.104493
f1                          0.169819
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jac 0.80

In [10]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.25]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.80, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.80, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 71.34it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [11:59, 13.33s/it]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 4h 33s, sys: 1min 9s, total: 4h 1min 42s
Wall time: 3h 21min 18s


In [11]:
len(all_refined_class_property_aligns_list)

21

In [16]:
all_class_property_exact_08 = pd.concat(all_class_property_evaluations_list)
all_class_property_exact_08

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd
0,7,5,5,1.000000,0.714286,0.833333,confof-sigkdd
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw
0,15,9,11,0.818182,0.600000,0.692308,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,10,0.500000,0.333333,0.400000,cmt-conference
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted
0,16,5,6,0.833333,0.312500,0.454545,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,10,11,0.909091,0.833333,0.869565,cmt-sigkdd


In [17]:
all_class_property_exact_08.to_csv("../data/conference/with_references/all_class_property_exact_08.csv")

In [13]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_1847/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.666667
numOfPredicted              9.238095
precision                   0.852450
recall                      0.563077
f1                          0.662833
dtype: float64

In [18]:
all_class_property_exact_08.sum()

numOfRefs                                                                305
numOfCorrectlyPredicted                                                  161
numOfPredicted                                                           194
precision                                                          17.901443
recall                                                             11.824612
f1                                                                 13.919483
test_case                  iasted-sigkddconfof-sigkddedas-ekawconference-...
dtype: object

In [32]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.25,0.087551,one-to-one
0,7,4,4,1.000000,0.571429,0.727273,confof-sigkdd,string_context_similarity,0.25,0.109206,one-to-one
0,23,10,13,0.769231,0.434783,0.555556,edas-ekaw,string_context_similarity,0.25,0.235250,one-to-one
0,15,8,9,0.888889,0.533333,0.666667,conference-sigkdd,string_context_similarity,0.25,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.25,0.081515,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.25,0.042640,one-to-one
0,4,4,5,0.800000,1.000000,0.888889,cmt-iasted,string_context_similarity,0.25,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.25,0.111429,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.25,0.037791,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.25,0.038805,one-to-one


In [33]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_80191/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.857143
numOfPredicted              7.809524
precision                   0.889611
recall                      0.505198
f1                          0.631327
similarity_threshold        0.250000
similarity_cutoff           0.083662
dtype: float64

In [34]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,0,1,0.0,0.000000,0.000000,cmt-conference
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd
0,13,2,2,1.0,0.153846,0.266667,cmt-edas
0,25,0,1,0.0,0.000000,0.000000,conference-ekaw


In [35]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_80191/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  16.250000
numOfCorrectlyPredicted     0.750000
numOfPredicted              1.250000
precision                   0.500000
recall                      0.059295
f1                          0.105128
dtype: float64

In [36]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,1,1.000000,0.142857,0.250000,confof-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,conference-sigkdd
0,15,1,3,0.333333,0.066667,0.111111,cmt-conference
0,16,1,1,1.000000,0.062500,0.117647,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,2,3,0.666667,0.133333,0.222222,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,3,0.000000,0.000000,0.000000,cmt-edas


In [37]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_80191/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.400000
numOfPredicted              2.500000
precision                   0.616667
recall                      0.097827
f1                          0.166711
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jac 0.85

In [233]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.25]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.85, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.85, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 69.20it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [12:02, 13.37s/it]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 3h 59min 33s, sys: 1min 11s, total: 4h 44s
Wall time: 3h 20min 51s


In [234]:
len(all_refined_class_property_aligns_list)

21

In [235]:
pd.concat(all_class_property_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd
0,7,5,6,0.833333,0.714286,0.769231,confof-sigkdd
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw
0,15,9,12,0.750000,0.600000,0.666667,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,10,0.500000,0.333333,0.400000,cmt-conference
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted
0,16,5,6,0.833333,0.312500,0.454545,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,10,11,0.909091,0.833333,0.869565,cmt-sigkdd


In [236]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_60200/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.619048
numOfPredicted              9.476190
precision                   0.814037
recall                      0.561006
f1                          0.646937
dtype: float64

In [237]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.62,0.274400,one-to-one
0,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd,string_context_similarity,0.62,0.109206,one-to-one
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw,string_context_similarity,0.62,0.619406,one-to-one
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd,string_context_similarity,0.62,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.62,0.265139,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.62,0.559763,one-to-one
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted,string_context_similarity,0.62,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.62,0.527445,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.62,0.316072,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.62,0.571298,one-to-one


In [238]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_60200/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              8.047619
precision                   0.849485
recall                      0.503128
f1                          0.615739
similarity_threshold        0.620000
similarity_cutoff           0.357616
dtype: float64

In [239]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,0,1,0.0,0.000000,0.000000,cmt-conference
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd
0,13,2,2,1.0,0.153846,0.266667,cmt-edas
0,25,0,1,0.0,0.000000,0.000000,conference-ekaw


In [240]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_60200/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  16.250000
numOfCorrectlyPredicted     0.750000
numOfPredicted              1.250000
precision                   0.500000
recall                      0.059295
f1                          0.105128
dtype: float64

In [241]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,1,1.000000,0.142857,0.250000,confof-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,conference-sigkdd
0,15,1,3,0.333333,0.066667,0.111111,cmt-conference
0,16,1,1,1.000000,0.062500,0.117647,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,2,3,0.666667,0.133333,0.222222,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,3,0.000000,0.000000,0.000000,cmt-edas


In [242]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_60200/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.400000
numOfPredicted              2.500000
precision                   0.616667
recall                      0.097827
f1                          0.166711
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jac 0.9

In [243]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.62]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.90, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.90, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 61.72it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [11:59, 13.32s/it]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 4h 36s, sys: 1min 12s, total: 4h 1min 49s
Wall time: 3h 21min 46s


In [244]:
len(all_refined_class_property_aligns_list)

21

In [245]:
pd.concat(all_class_property_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd
0,7,5,6,0.833333,0.714286,0.769231,confof-sigkdd
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw
0,15,9,12,0.750000,0.600000,0.666667,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,9,0.555556,0.333333,0.416667,cmt-conference
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted
0,16,5,6,0.833333,0.312500,0.454545,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,10,11,0.909091,0.833333,0.869565,cmt-sigkdd


In [246]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_60200/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.476190
numOfPredicted              9.238095
precision                   0.816510
recall                      0.550506
f1                          0.640885
dtype: float64

In [247]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.62,0.274400,one-to-one
0,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd,string_context_similarity,0.62,0.109206,one-to-one
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw,string_context_similarity,0.62,0.619406,one-to-one
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd,string_context_similarity,0.62,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.62,0.265139,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.62,0.559763,one-to-one
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted,string_context_similarity,0.62,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.62,0.527445,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.62,0.316072,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.62,0.571298,one-to-one


In [248]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_60200/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              8.047619
precision                   0.849485
recall                      0.503128
f1                          0.615739
similarity_threshold        0.620000
similarity_cutoff           0.357616
dtype: float64

In [249]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd


In [250]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_60200/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  12.000000
numOfCorrectlyPredicted     1.000000
numOfPredicted              1.000000
precision                   1.000000
recall                      0.083333
f1                          0.153846
dtype: float64

In [251]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,1,1.000000,0.142857,0.250000,confof-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,conference-sigkdd
0,15,1,3,0.333333,0.066667,0.111111,cmt-conference
0,16,1,1,1.000000,0.062500,0.117647,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,1,2,0.500000,0.066667,0.117647,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,3,0.000000,0.000000,0.000000,cmt-edas


In [252]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_60200/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.300000
numOfPredicted              2.400000
precision                   0.600000
recall                      0.091160
f1                          0.156254
dtype: float64

# Class Matching + Exact Property Matchings for Individual Conference Cases + Jac 0.95

In [253]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.62]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.95, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.95, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 59.38it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [11:40, 12.98s/it]

21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 4h 2min 45s, sys: 1min 15s, total: 4h 4min 1s
Wall time: 3h 23min 19s


In [254]:
len(all_refined_class_property_aligns_list)

21

In [255]:
pd.concat(all_class_property_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd
0,7,5,6,0.833333,0.714286,0.769231,confof-sigkdd
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw
0,15,9,12,0.750000,0.600000,0.666667,conference-sigkdd
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted
0,15,5,8,0.625000,0.333333,0.434783,cmt-conference
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted
0,16,5,6,0.833333,0.312500,0.454545,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,10,11,0.909091,0.833333,0.869565,cmt-sigkdd


In [256]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_60200/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     7.476190
numOfPredicted              9.142857
precision                   0.823280
recall                      0.550506
f1                          0.643128
dtype: float64

In [257]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.62,0.274400,one-to-one
0,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd,string_context_similarity,0.62,0.109206,one-to-one
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw,string_context_similarity,0.62,0.619406,one-to-one
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd,string_context_similarity,0.62,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.62,0.265139,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.62,0.559763,one-to-one
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted,string_context_similarity,0.62,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.62,0.527445,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.62,0.316072,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.62,0.571298,one-to-one


In [258]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_60200/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              8.047619
precision                   0.849485
recall                      0.503128
f1                          0.615739
similarity_threshold        0.620000
similarity_cutoff           0.357616
dtype: float64

In [259]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,12,1,1,1.0,0.083333,0.153846,cmt-sigkdd


In [260]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_60200/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  12.000000
numOfCorrectlyPredicted     1.000000
numOfPredicted              1.000000
precision                   1.000000
recall                      0.083333
f1                          0.153846
dtype: float64

In [261]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,1,1.000000,0.142857,0.250000,confof-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,conference-sigkdd
0,15,1,2,0.500000,0.066667,0.117647,cmt-conference
0,16,1,1,1.000000,0.062500,0.117647,cmt-confof
0,12,1,2,0.500000,0.083333,0.142857,cmt-sigkdd
0,19,2,3,0.666667,0.105263,0.181818,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,1,2,0.500000,0.066667,0.117647,edas-sigkdd
0,17,2,4,0.500000,0.117647,0.190476,conference-edas
0,13,0,2,0.000000,0.000000,0.000000,cmt-edas


In [262]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_60200/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.400000
numOfCorrectlyPredicted     1.300000
numOfPredicted              2.200000
precision                   0.616667
recall                      0.091160
f1                          0.156907
dtype: float64

# Class Matching + Wasserstein Property Matchings for Individual Conference Cases

In [54]:
conf_folder = "../data/conference/with_references"
source_url = conf_folder + "/confof-edas/source.rdf"
target_url = conf_folder + '/confof-edas/target.rdf'
refs_url = conf_folder + '/confof-edas/reference.rdf'

subf = 'confof-edas'

source_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(source_url)
maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
#len(source_graph)

target_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(target_url)
maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
print("len(source_graph)={}, len(target_graph)={}".format(len(source_graph), len(target_graph)))
        
# Concept labels
slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
print("#slabel={}, #tlabel={}".format(slabel_clnd_uris.shape, tlabel_clnd_uris.shape))

sobject_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
tobject_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
print("#sobject={}, #tobject={}".format(sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))

sdatatype_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
tdatatype_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
print("#sdatatype={}, #tdatatype={}".format(sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))

../data/conference/with_references/confof-edas/source.rdf
../data/conference/with_references/confof-edas/target.rdf
len(source_graph)=539, len(target_graph)=1720
#slabel=(38, 3), #tlabel=(103, 3)
#sobject=(15, 2), #tobject=(48, 2)
#sdatatype=(32, 2), #tdatatype=(21, 2)


In [202]:
# compute property mathcing based on jaccard similarity of synonym phrases
def compute_property_synJac_matchings(sproperty_uris, tproperty_uris, threshold, subf):
    """
        input: sproperty_uris: a DataFrame containing source properties ['label', 'uri']
               tproperty_uris: a DataFrame containing target properties ['label', 'uri']
               subf: a string representing the test case
        output: a DataFrame containing candidate property matchings with
                ['source', 'target', 'test_case', 'sLabel', 'tLabel', 
                 'sDomainRangeLabel', 'tDomainRangeLabel',
                 'Jaccard_similarity', 'threshold']
    """
    
    property_matchings = []
    
    for i, srow in tqdm(sproperty_uris.iterrows()):
        # get the uri
        suri = srow['uri']
        sDomainRangeLabel = srow['label']

        # extract the raw label
        slabel = mapneighbor.extract_label_from_uri(suri)

        # clean the label by separating individual words using spaces
        sClndLabel = " ".join(maponto.clean_document_norm(slabel))
        # clean the label by separating individual words using spaces
        sClndDomainRangeLabel = " ".join(maponto.remove_duplicates(maponto.clean_document_keepStops(sDomainRangeLabel)))
        # clean the label by separating individual words using spaces
        sClndDomainRangeLabelNoStop = " ".join(maponto.remove_duplicates(maponto.clean_document_lower(sDomainRangeLabel)))

        #print(sClndLabel)
        #print(sClndDomainRangeLabel)
        #print(mapneighbor.get_syn_phrases(sClndDomainRangeLabelNoStop, 5))
        #print(sClndDomainRangeLabelNoStop)

        sSynLabels = mapneighbor.get_syn_phrases(sClndDomainRangeLabelNoStop, 5)
        
        for i, trow in tproperty_uris.iterrows():
            turi = trow['uri']
            tDomainRangeLabel = trow['label']


            # extract the raw label
            tlabel = mapneighbor.extract_label_from_uri(turi)


            # clean the label by separating individual words using spaces
            tClndLabel = " ".join(maponto.clean_document_norm(tlabel))
            # clean the label by separating individual words using spaces
            tClndDomainRangeLabel = " ".join(maponto.remove_duplicates(maponto.clean_document_keepStops(tDomainRangeLabel)))
            # clean the label by separating individual words using spaces
            tClndDomainRangeLabelNoStop = " ".join(maponto.remove_duplicates(maponto.clean_document_lower(tDomainRangeLabel)))

            #print(tClndLabel)
            #print(tClndDomainRangeLabel)
            #print(mapneighbor.get_syn_phrases(tClndDomainRangeLabelNoStop, 5))
            #print(tClndDomainRangeLabelNoStop)

            tSynLabels = mapneighbor.get_syn_phrases(tClndDomainRangeLabelNoStop, 5)
            
            mapped = False
    
            for sSynLabel in sSynLabels:
                for tSynLabel in tSynLabels:
                    jacSim = mapneighbor.jaccard_similarity(sSynLabel.split(), 
                                                            tSynLabel.split())
                    #print(jacSim)
                    if (jacSim >= threshold):
                        amap = {'source':suri, 'target':turi, 
                                'test_case':subf,
                                'sLabel':slabel,
                                'tLabel':tlabel,
                                'sDomainRangeLabel':sDomainRangeLabel,
                                'tDomainRangeLabel':tDomainRangeLabel,
                                'Jaccard_similarity': jacSim,
                                'threshold':threshold}
                        
                        mapped = True
                        property_matchings.append(amap)
                        break
                if mapped:
                    break
    
    aligns = pd.DataFrame(property_matchings)
    
    if len(aligns) > 0:
        aligns = aligns.sort_values('Jaccard_similarity', ascending=False)
    
        idx = []
        sources = []
        targets = []
        for j, align in aligns.iterrows():
            if (align['source'] not in sources) & (align['target'] not in targets):
                sources.append(align['source'])
                targets.append(align['target'])
                idx.append(j)

        return aligns.loc[idx].reset_index(drop=True)
    else:
        return pd.DataFrame(columns=['source', 'target'])

In [174]:
aligns = compute_property_synJac_matchings(sdatatype_uris, tdatatype_uris, 0.5, subf)
aligns

32it [00:56,  1.77s/it]


,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://confOf#hasEmail,http://edas#hasEmail,confof-edas,hasEmail,hasEmail,Person hasEmail,Person hasEmail,1.0,0.5
1,http://confOf#hasFirstName,http://edas#hasFirstName,confof-edas,hasFirstName,hasFirstName,Person hasFirstName,Person hasFirstName,1.0,0.5
2,http://confOf#hasPostalCode,http://edas#hasPostalCode,confof-edas,hasPostalCode,hasPostalCode,Person hasPostalCode,ContactInformation hasPostalCode,0.5,0.5
3,http://confOf#hasSurname,http://edas#hasLastName,confof-edas,hasSurname,hasLastName,Person hasSurname,Person hasLastName,0.5,0.5


In [177]:
aligns_evaluation = maponto.evaluate_noprint(aligns, refs_url)
aligns_evaluation['test_case']= subf
pd.DataFrame([aligns_evaluation])                

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,19,2,4,0.5,0.105263,0.173913,confof-edas


In [180]:
aligns = compute_property_synJac_matchings(sobject_uris, tobject_uris, 0.5, subf)
aligns

15it [00:14,  1.07it/s]


,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://confOf#writtenBy,http://edas#isWrittenBy,confof-edas,writtenBy,isWrittenBy,Contribution writtenBy Author,Paper isWrittenBy Author,0.5,0.5


In [181]:
aligns_evaluation = maponto.evaluate_noprint(aligns, refs_url)
aligns_evaluation['test_case']= subf
pd.DataFrame([aligns_evaluation])    

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,19,1,1,1.0,0.052632,0.1,confof-edas


In [203]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.62]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            refined_object_aligns = compute_property_synJac_matchings(sobject_domain_range_uris, 
                                                              tobject_domain_range_uris, 0.5, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            refined_datatype_aligns = compute_property_synJac_matchings(sdatatype_domain_uris, 
                                                              tdatatype_domain_uris, 0.5, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 63.36it/s]


21. #sobject=(54, 2), #tobject=(33, 2)


54it [11:45, 13.06s/it]


21. #sdatatype=(20, 2), #tdatatype=(0, 2)


In [204]:
len(all_refined_class_property_aligns_list)

21

In [205]:
pd.concat(all_class_property_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,12,14,0.857143,0.800000,0.827586,iasted-sigkdd
0,7,5,7,0.714286,0.714286,0.714286,confof-sigkdd
0,23,12,22,0.545455,0.521739,0.533333,edas-ekaw
0,15,9,14,0.642857,0.600000,0.620690,conference-sigkdd
0,19,8,10,0.800000,0.421053,0.551724,edas-iasted
0,15,5,14,0.357143,0.333333,0.344828,cmt-conference
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted
0,16,6,8,0.750000,0.375000,0.500000,cmt-confof
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw
0,12,10,12,0.833333,0.833333,0.833333,cmt-sigkdd


In [206]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_60200/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     8.000000
numOfPredicted             11.904762
precision                   0.707852
recall                      0.585199
f1                          0.624651
dtype: float64

In [207]:
pd.concat(all_class_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.62,0.274400,one-to-one
0,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd,string_context_similarity,0.62,0.109206,one-to-one
0,23,9,12,0.750000,0.391304,0.514286,edas-ekaw,string_context_similarity,0.62,0.619406,one-to-one
0,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd,string_context_similarity,0.62,0.116958,one-to-one
0,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.62,0.265139,one-to-one
0,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.62,0.559763,one-to-one
0,4,4,6,0.666667,1.000000,0.800000,cmt-iasted,string_context_similarity,0.62,0.061687,one-to-one
0,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.62,0.527445,one-to-one
0,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.62,0.316072,one-to-one
0,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.62,0.571298,one-to-one


In [208]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_60200/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              8.047619
precision                   0.849485
recall                      0.503128
f1                          0.615739
similarity_threshold        0.620000
similarity_cutoff           0.357616
dtype: float64

In [209]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,0,1,0.000000,0.000000,0.000000,iasted-sigkdd
0,23,3,10,0.300000,0.130435,0.181818,edas-ekaw
0,15,0,4,0.000000,0.000000,0.000000,cmt-conference
0,12,1,1,1.000000,0.083333,0.153846,cmt-sigkdd
0,19,1,1,1.000000,0.052632,0.100000,confof-edas
0,11,3,4,0.750000,0.272727,0.400000,cmt-ekaw
0,9,0,2,0.000000,0.000000,0.000000,confof-iasted
0,15,0,1,0.000000,0.000000,0.000000,edas-sigkdd
0,17,0,5,0.000000,0.000000,0.000000,conference-edas
0,13,1,9,0.111111,0.076923,0.090909,cmt-edas


In [210]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_60200/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  15.818182
numOfCorrectlyPredicted     0.818182
numOfPredicted              3.727273
precision                   0.287374
recall                      0.056005
f1                          0.084234
dtype: float64

In [211]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,2,0.500000,0.142857,0.222222,confof-sigkdd
0,15,1,4,0.250000,0.066667,0.105263,conference-sigkdd
0,19,0,1,0.000000,0.000000,0.000000,edas-iasted
0,15,1,4,0.250000,0.066667,0.105263,cmt-conference
0,16,2,3,0.666667,0.125000,0.210526,cmt-confof
0,12,1,3,0.333333,0.083333,0.133333,cmt-sigkdd
0,19,2,4,0.500000,0.105263,0.173913,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,3,5,0.600000,0.200000,0.300000,edas-sigkdd
0,17,2,7,0.285714,0.117647,0.166667,conference-edas


In [212]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_60200/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.818182
numOfCorrectlyPredicted     1.454545
numOfPredicted              3.636364
precision                   0.398701
recall                      0.100676
f1                          0.159138
dtype: float64

# Ensemble Class Matching For Individual Conference Cases

In [43]:
importlib.reload(maponto)
importlib.reload(mapneighbor)
importlib.reload(otmapper)

INFO:root:logging info test


<module 'OTMapper' from '/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapper.py'>

In [95]:
%%time

import time
import os

all_class_candidates_list = []
all_class_candidates_evaluation_list = []

all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.62]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        class_candidates = otmapper.ensemble_class_map(source_url, target_url, embs_model)
    
        
        # before refinement    
        all_class_candidates_evaluation = maponto.evaluate_noprint(class_candidates, refs_url)
        all_class_candidates_evaluation['test_case']= subf  
        
        all_class_candidates_list.append(class_candidates)
        all_class_candidates_evaluation_list.append(all_class_candidates_evaluation)
        

INFO:root:Python ensemble class mapper info: map ../data/conference/with_references/iasted-sigkdd/source.rdf to ../data/conference/with_references/iasted-sigkdd/target.rdf


../data/conference/with_references/iasted-sigkdd/source.rdf


INFO:root:Read source with 1173 triples.
INFO:root:Read target with 245 triples.
INFO:root:MAP CONCEPTS


../data/conference/with_references/iasted-sigkdd/target.rdf


INFO:root:The number of level_1 predicted mapping is 12.
/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapper.py:1110: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  ensemble_align = pd.concat([ensemble_align, current_align], 0)
INFO:root:match_label_synonyms(slabel_clnd_uris_sub, tlabel_clnd_uris_sub, input_alignment): Retrieving Synsets by WordNet...
128it [00:00, 1258.06it/s]
37it [00:00, 2876.43it/s]
INFO:root:Compute the related words for each source concept....
128it [00:05, 22.80it/s]
INFO:root:Total number of source concepts computed with relatd words is 128
INFO:root:Compute the related words for each target concept....
37it [00:01, 26.65it/s]
INFO:root:Total number of target concepts computed with related words is 37
100%|██████████| 128/128 [00:00<00:00, 5621.74it/s]
INFO:root:The number of level_3 predicted mapping is 12
/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapper.

...

../data/conference/with_references/cmt-edas/source.rdf
../data/conference/with_references/cmt-edas/target.rdf


INFO:root:Read target with 1720 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 8.
/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapper.py:1110: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  ensemble_align = pd.concat([ensemble_align, current_align], 0)
INFO:root:match_label_synonyms(slabel_clnd_uris_sub, tlabel_clnd_uris_sub, input_alignment): Retrieving Synsets by WordNet...
21it [00:00, 1174.20it/s]
95it [00:00, 2115.41it/s]
INFO:root:Compute the related words for each source concept....
21it [00:00, 25.75it/s]
INFO:root:Total number of source concepts computed with relatd words is 21
INFO:root:Compute the related words for each target concept....
95it [00:03, 26.85it/s]
INFO:root:Total number of target concepts computed with related words is 95
100%|██████████| 21/21 [00:00<00:00, 2015.52it/s]
INFO:root:The number of level_3 predicted mapping is 5
/home/ubu

../data/conference/with_references/conference-ekaw/source.rdf
../data/conference/with_references/conference-ekaw/target.rdf


INFO:root:Read target with 557 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 12.
/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapper.py:1110: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  ensemble_align = pd.concat([ensemble_align, current_align], 0)
INFO:root:match_label_synonyms(slabel_clnd_uris_sub, tlabel_clnd_uris_sub, input_alignment): Retrieving Synsets by WordNet...
47it [00:00, 1883.66it/s]
61it [00:00, 2266.89it/s]
INFO:root:The number of level_2 predicted mapping is 3
/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapper.py:1119: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  ensemble_align = pd.concat([ensemble_align, current_align], 0)
INFO:root:Compute the related words for each source concept....
44it [00:01, 27.87it/s]
INFO:root:Total number of source conc

CPU times: user 3min 23s, sys: 1.72 s, total: 3min 25s
Wall time: 3min 24s


In [96]:
len(all_class_candidates_evaluation_list)

21

In [97]:
pd.DataFrame(all_class_candidates_evaluation_list).mean()

/tmp/ipykernel_99446/1565320118.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.DataFrame(all_class_candidates_evaluation_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     9.523810
numOfPredicted             27.476190
precision                   0.360101
recall                      0.678392
f1                          0.454728
dtype: float64

In [98]:
import pickle
with open("../data/conference/with_references/preds_candidates_ensemble_OT_list.pk", "wb") as file:
    pickle.dump(all_class_candidates_list, file)

# Combine preds_topn_refined with preds_ensemble_OT

In [99]:
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)
all_refs = []
for subf in subfolders:
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(refs_url):
        all_refs.append(refs_url)

In [101]:
with open("../data/conference/with_references/preds_candidates_ensemble_OT_list.pk", "rb") as file:
    preds_candidates_ensemble_OT_list = pickle.load(file)

In [103]:
evaluations = []
for i, candidates in enumerate(preds_candidates_ensemble_OT_list):
    evaluation = maponto.evaluate_noprint(candidates, all_refs[i])
    evaluations.append(evaluation)

In [104]:
pd.DataFrame(evaluations).mean()

numOfRefs                  14.523810
numOfCorrectlyPredicted     9.523810
numOfPredicted             27.476190
precision                   0.360101
recall                      0.678392
f1                          0.454728
dtype: float64

In [105]:
with open("../data/conference/with_references/all_refined_class_aligns_list.pk", "rb") as file:
    preds_candidates_topn_refined_list = pickle.load(file)

In [107]:
evaluations = []
for i, candidates in enumerate(preds_candidates_topn_refined_list):
    evaluation = maponto.evaluate_noprint(candidates, all_refs[i])
    evaluations.append(evaluation)

In [108]:
pd.DataFrame(evaluations).mean()

numOfRefs                  14.523810
numOfCorrectlyPredicted     6.857143
numOfPredicted              7.809524
precision                   0.889611
recall                      0.505198
f1                          0.631327
dtype: float64

In [109]:
preds_candidates_topn_refined_list[3].columns

Index(['source', 'source_label_x', 'target', 'target_label_x',
       'target_label_y', 'source_label_y', 'string_similarity',
       'context_similarity', 'label_embeddings_similarity',
       'label_embeddings_wd_similarity', 'string_context_similarity',
       'label_emb_context_similarity', 'label_emb_wd_context_similarity',
       'label_emb_wd_context_all_similarity'],
      dtype='object')

In [121]:
# join topn_refined with ensemble_OT
evaluations = []
for i, topn in enumerate(preds_candidates_topn_refined_list):
    ensembleOT = preds_candidates_ensemble_OT_list[i]
    candidates = topn.merge(ensembleOT, on=['source', 'target'], how='left')
    evaluation = maponto.evaluate_noprint(candidates, all_refs[i])
    evaluations.append(evaluation)

In [122]:
pd.DataFrame(evaluations).mean()

numOfRefs                  14.523810
numOfCorrectlyPredicted     6.857143
numOfPredicted              7.809524
precision                   0.889611
recall                      0.505198
f1                          0.631327
dtype: float64

# Class Matching + Wasserstein Property Matchings for Individual Conference Cases

In [54]:
conf_folder = "../data/conference/with_references"
source_url = conf_folder + "/confof-edas/source.rdf"
target_url = conf_folder + '/confof-edas/target.rdf'
refs_url = conf_folder + '/confof-edas/reference.rdf'

subf = 'confof-edas'

source_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(source_url)
maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
#len(source_graph)

target_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(target_url)
maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
print("len(source_graph)={}, len(target_graph)={}".format(len(source_graph), len(target_graph)))
        
# Concept labels
slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
print("#slabel={}, #tlabel={}".format(slabel_clnd_uris.shape, tlabel_clnd_uris.shape))

sobject_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
tobject_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
print("#sobject={}, #tobject={}".format(sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))

sdatatype_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
tdatatype_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
print("#sdatatype={}, #tdatatype={}".format(sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))

../data/conference/with_references/confof-edas/source.rdf
../data/conference/with_references/confof-edas/target.rdf
len(source_graph)=539, len(target_graph)=1720
#slabel=(38, 3), #tlabel=(103, 3)
#sobject=(15, 2), #tobject=(48, 2)
#sdatatype=(32, 2), #tdatatype=(21, 2)


In [202]:
# compute property mathcing based on jaccard similarity of synonym phrases
def compute_property_synJac_matchings(sproperty_uris, tproperty_uris, threshold, subf):
    """
        input: sproperty_uris: a DataFrame containing source properties ['label', 'uri']
               tproperty_uris: a DataFrame containing target properties ['label', 'uri']
               subf: a string representing the test case
        output: a DataFrame containing candidate property matchings with
                ['source', 'target', 'test_case', 'sLabel', 'tLabel', 
                 'sDomainRangeLabel', 'tDomainRangeLabel',
                 'Jaccard_similarity', 'threshold']
    """
    
    property_matchings = []
    
    for i, srow in tqdm(sproperty_uris.iterrows()):
        # get the uri
        suri = srow['uri']
        sDomainRangeLabel = srow['label']

        # extract the raw label
        slabel = mapneighbor.extract_label_from_uri(suri)

        # clean the label by separating individual words using spaces
        sClndLabel = " ".join(maponto.clean_document_norm(slabel))
        # clean the label by separating individual words using spaces
        sClndDomainRangeLabel = " ".join(maponto.remove_duplicates(maponto.clean_document_keepStops(sDomainRangeLabel)))
        # clean the label by separating individual words using spaces
        sClndDomainRangeLabelNoStop = " ".join(maponto.remove_duplicates(maponto.clean_document_lower(sDomainRangeLabel)))

        #print(sClndLabel)
        #print(sClndDomainRangeLabel)
        #print(mapneighbor.get_syn_phrases(sClndDomainRangeLabelNoStop, 5))
        #print(sClndDomainRangeLabelNoStop)

        sSynLabels = mapneighbor.get_syn_phrases(sClndDomainRangeLabelNoStop, 5)
        
        for i, trow in tproperty_uris.iterrows():
            turi = trow['uri']
            tDomainRangeLabel = trow['label']


            # extract the raw label
            tlabel = mapneighbor.extract_label_from_uri(turi)


            # clean the label by separating individual words using spaces
            tClndLabel = " ".join(maponto.clean_document_norm(tlabel))
            # clean the label by separating individual words using spaces
            tClndDomainRangeLabel = " ".join(maponto.remove_duplicates(maponto.clean_document_keepStops(tDomainRangeLabel)))
            # clean the label by separating individual words using spaces
            tClndDomainRangeLabelNoStop = " ".join(maponto.remove_duplicates(maponto.clean_document_lower(tDomainRangeLabel)))

            #print(tClndLabel)
            #print(tClndDomainRangeLabel)
            #print(mapneighbor.get_syn_phrases(tClndDomainRangeLabelNoStop, 5))
            #print(tClndDomainRangeLabelNoStop)

            tSynLabels = mapneighbor.get_syn_phrases(tClndDomainRangeLabelNoStop, 5)
            
            mapped = False
    
            for sSynLabel in sSynLabels:
                for tSynLabel in tSynLabels:
                    jacSim = mapneighbor.jaccard_similarity(sSynLabel.split(), 
                                                            tSynLabel.split())
                    #print(jacSim)
                    if (jacSim >= threshold):
                        amap = {'source':suri, 'target':turi, 
                                'test_case':subf,
                                'sLabel':slabel,
                                'tLabel':tlabel,
                                'sDomainRangeLabel':sDomainRangeLabel,
                                'tDomainRangeLabel':tDomainRangeLabel,
                                'Jaccard_similarity': jacSim,
                                'threshold':threshold}
                        
                        mapped = True
                        property_matchings.append(amap)
                        break
                if mapped:
                    break
    
    aligns = pd.DataFrame(property_matchings)
    
    if len(aligns) > 0:
        aligns = aligns.sort_values('Jaccard_similarity', ascending=False)
    
        idx = []
        sources = []
        targets = []
        for j, align in aligns.iterrows():
            if (align['source'] not in sources) & (align['target'] not in targets):
                sources.append(align['source'])
                targets.append(align['target'])
                idx.append(j)

        return aligns.loc[idx].reset_index(drop=True)
    else:
        return pd.DataFrame(columns=['source', 'target'])

In [174]:
aligns = compute_property_synJac_matchings(sdatatype_uris, tdatatype_uris, 0.5, subf)
aligns

32it [00:56,  1.77s/it]


,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://confOf#hasEmail,http://edas#hasEmail,confof-edas,hasEmail,hasEmail,Person hasEmail,Person hasEmail,1.0,0.5
1,http://confOf#hasFirstName,http://edas#hasFirstName,confof-edas,hasFirstName,hasFirstName,Person hasFirstName,Person hasFirstName,1.0,0.5
2,http://confOf#hasPostalCode,http://edas#hasPostalCode,confof-edas,hasPostalCode,hasPostalCode,Person hasPostalCode,ContactInformation hasPostalCode,0.5,0.5
3,http://confOf#hasSurname,http://edas#hasLastName,confof-edas,hasSurname,hasLastName,Person hasSurname,Person hasLastName,0.5,0.5


In [177]:
aligns_evaluation = maponto.evaluate_noprint(aligns, refs_url)
aligns_evaluation['test_case']= subf
pd.DataFrame([aligns_evaluation])                

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,19,2,4,0.5,0.105263,0.173913,confof-edas


In [180]:
aligns = compute_property_synJac_matchings(sobject_uris, tobject_uris, 0.5, subf)
aligns

15it [00:14,  1.07it/s]


,source,target,test_case,sLabel,tLabel,sDomainRangeLabel,tDomainRangeLabel,Jaccard_similarity,threshold
0,http://confOf#writtenBy,http://edas#isWrittenBy,confof-edas,writtenBy,isWrittenBy,Contribution writtenBy Author,Paper isWrittenBy Author,0.5,0.5


In [181]:
aligns_evaluation = maponto.evaluate_noprint(aligns, refs_url)
aligns_evaluation['test_case']= subf
pd.DataFrame([aligns_evaluation])    

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,19,1,1,1.0,0.052632,0.1,confof-edas


In [44]:
%%time

import time
import os
all_refined_class_aligns_list = []
all_class_evaluations_list = []

all_refined_object_aligns_list = []
all_object_evaluations_list = []

all_refined_datatype_aligns_list = []
all_datatype_evaluations_list = []

all_refined_class_property_aligns_list = []
all_class_property_evaluations_list = []

## for each conference case
case_index = 0
count = 0
conf_folder = "../data/conference/with_references"
subfolders = os.listdir(conf_folder)

criteria_class = ['string_context_similarity']
thresholds_class = [0.62]
matching_pairing_class = ['one-to-one']

criteria_object = ['stDRLabEmbSim']
thresholds_object = [0.98]
matching_pairing_object = ['one-to-one']

criteria_datatype = ['stDRLabNSSim']
thresholds_datatype = [0.55]
matching_pairing_datatype = ['one-to-one']

for subf in subfolders:
        
    source_url = conf_folder + "/" + subf + "/source.rdf"
    target_url = conf_folder + '/' + subf + '/target.rdf'
    refs_url = conf_folder + '/' + subf + '/reference.rdf'
    if os.path.exists(source_url) and os.path.exists(target_url) and os.path.exists(refs_url):
        case_index = case_index + 1
        if case_index > 25:
            break
        
        # extract RDF graphs forom rdf files
        source_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(source_url)
        maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
        #len(source_graph)

        target_graph = Graph()
        stack_urls = []
        visited_urls = []
        stack_urls.append(target_url)
        maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
        
        #time.sleep(0.5)
        print("{}. len(source_graph)={}, len(target_graph)={}".format(case_index, len(source_graph), len(target_graph)))
        
        
        ## Find matchings between classes
        
        # Concept labels
        slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
        tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
        print("{} #slabel={}, #tlabel={}".format(case_index, slabel_clnd_uris.shape, tlabel_clnd_uris.shape))
        
        # extract relation edges
        sedges = maponto.build_relation_graph_edges_df(source_graph)
        tedges = maponto.build_relation_graph_edges_df(target_graph)
        print("{}. #sedges={}, #tedges={}".format(case_index, sedges.shape, tedges.shape))
        
        class_topn_candidates, all_results = maponto.compute_topn_candidates(subf, 
                                        slabel_clnd_uris, tlabel_clnd_uris, source_graph, 
                                         target_graph, refs_url, embs_model)
        
        class_similarities = maponto.compute_candidate_similarities(class_topn_candidates, sedges, 
                                                              tedges, embs_model)
        
        class_topn_candidates['string_similarity'] = \
            class_similarities['string_similarity']
        class_topn_candidates['context_similarity'] = \
            class_similarities['context_similarity']
        class_topn_candidates['label_embeddings_similarity'] = \
            class_similarities['label_embeddings_similarity']
        class_topn_candidates['label_embeddings_wd_similarity'] = \
            class_similarities['label_embeddings_wd_similarity']
        class_topn_candidates['string_context_similarity'] = \
            class_similarities['string_context_similarity']
        class_topn_candidates['label_emb_context_similarity'] = \
            class_similarities['label_emb_context_similarity']
        class_topn_candidates['label_emb_wd_context_similarity'] = \
            class_similarities['label_emb_wd_context_similarity']
        class_topn_candidates['label_emb_wd_context_all_similarity'] = \
            class_similarities['label_emb_wd_context_all_similarity']
            
        
        all_class_evaluations, refined_class_aligns = maponto.refine_candidate_matchings(class_topn_candidates, 
                                                    criteria_class, thresholds_class, 
                                                    matching_pairing_class, subf, refs_url)
        
        all_refined_class_aligns_list.append(refined_class_aligns)
        all_class_evaluations_list.append(all_class_evaluations)
        
        ## Find matchings between object properties
        
        # extract object properties
        sobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(source_graph)
        tobject_domain_range_uris = mapneighbor.extract_objectProperty_domain_range_uris(target_graph)
        print("{}. #sobject={}, #tobject={}".format(case_index, sobject_domain_range_uris.shape, 
                                                    tobject_domain_range_uris.shape))
        
        # initialize an empty objectProperty matchings
        refined_object_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sobject_domain_range_uris) != 0) & (len(tobject_domain_range_uris) != 0):
            #refined_object_aligns = compute_property_synJac_matchings(sobject_domain_range_uris, 
             #                                                 tobject_domain_range_uris, 0.5, subf)
        
            if len(refined_object_aligns) > 0:
                object_evaluation = maponto.evaluate_noprint(refined_object_aligns, refs_url)
            
                object_evaluation['test_case']= subf
                all_object_evaluations = pd.DataFrame([object_evaluation])     
            
                all_refined_object_aligns_list.append(refined_object_aligns)
                all_object_evaluations_list.append(all_object_evaluations)
            else:
                all_refined_object_aligns_list.append(None)
                all_object_evaluations_list.append(None)
        else:
            # if either source or target object list is empty, create an empty DataFrame with
            # the same columns (see the next for Datatype matching) or pd.DataFrame() or None
            all_refined_object_aligns_list.append(None)
            all_object_evaluations_list.append(None)
            
        
        ## Find matchings between datatype properties
        
        # extract datatype properties
        sdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(source_graph)
        tdatatype_domain_uris = mapneighbor.extract_datatypeProperty_domain_uris(target_graph)
        print("{}. #sdatatype={}, #tdatatype={}".format(case_index, sdatatype_domain_uris.shape, 
                                                    tdatatype_domain_uris.shape))
        
        # initialize an empty datatypeProperty matchings
        refined_datatype_aligns = pd.DataFrame(columns=['source', 'target'])
        
        if (len(sdatatype_domain_uris) != 0) & (len(tdatatype_domain_uris) != 0):
            #refined_datatype_aligns = compute_property_synJac_matchings(sdatatype_domain_uris, 
             #                                                 tdatatype_domain_uris, 0.5, subf)
            if len(refined_datatype_aligns) > 0:
        
                datatype_evaluation = maponto.evaluate_noprint(refined_datatype_aligns, refs_url)
            
                datatype_evaluation['test_case']= subf
                all_datatype_evaluations = pd.DataFrame([datatype_evaluation])
            
                all_refined_datatype_aligns_list.append(refined_datatype_aligns)
                all_datatype_evaluations_list.append(all_datatype_evaluations)
            else:
                all_refined_datatype_aligns_list.append(None)
                all_datatype_evaluations_list.append(None)
        else:
            all_refined_datatype_aligns_list.append(None)
            all_datatype_evaluations_list.append(None)
        
    
        #### combine class, object, and datatype aligns and evaluation results
        aligns_class = refined_class_aligns[['source', 'target']]
        aligns_object = refined_object_aligns[['source', 'target']]
        aligns_datatype = refined_datatype_aligns[['source', 'target']]
        
        aligns_class_property = pd.concat([aligns_class, aligns_object, aligns_datatype], axis=0)
        evaluation_aligns_class_property = maponto.evaluate_noprint(aligns_class_property, refs_url)
        evaluation_aligns_class_property['test_case']= subf
            
        all_refined_class_property_aligns_list.append(aligns_class_property)
        all_class_property_evaluations_list.append(pd.DataFrame([evaluation_aligns_class_property]))

../data/conference/with_references/iasted-sigkdd/source.rdf
../data/conference/with_references/iasted-sigkdd/target.rdf
1. len(source_graph)=1173, len(target_graph)=245


INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....


1 #slabel=(140, 3), #tlabel=(49, 3)


INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:build_relation_graph_edges_df(rdfGraph): Building a DataFrame containing the edges of a relation graph from the given RDF triple graph...
INFO:root:build_comments_edges()....
INFO:root:build_datatype_property_edges()....
INFO:root:build_disjoint_edges()....
INFO:root:build_equivalentClass_edges()....
INFO:root:build_relations_allValuesFrom_edges()....
INFO:root:build_relations_asDomains_edges()....
INFO:root:build_relations_maxCardinality_edges()....
INFO:root:build_relations_minCardinality_edges()....
INFO:root:build_relations_someValuesFrom_edges()....
INFO:root:build_subClassOf_edges()....
INFO:root:retrieve_hasRelatedSynonyms(graph): No oboInOwl:hasRelatedSynonym in this ontology

1. #sedges=(317, 4), #tedges=(83, 4)
match_concepts_labels: 12
match_concept_relatedSynonym_labels: 0


...

match_label_synonyms: 3


INFO:root:match_label_embeddings_OT(slabel_clnd_uris_sub, tlabel_clnd_uris_sub, embs_model,                         make_mappings_func, top_n, input_alignment): Matching Label Embeddings by Optimal Transport...
INFO:root:costs_embeddings(src_label_list, tgt_label_list, embs_model): Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:costs_embeddings(src_label_list, tgt_label_list, embs_model):The shape of the cost matrix is (58, 44)
INFO:root:ot_couplings(lmv, lnv, costs): Computing Optimal Transport Plan...
INFO:root:ot_couplings(lmv, lnv, costs): Computing Wasserstein distance by the Sinkhorn algorithm...
INFO:root:make_mappings_topn(): Making Top_n Mappings from the Optimal Transport Plan...
1274it [00:08, 143.17it/s]


string_context_similarity
one-to-one


100%|██████████| 1/1 [00:00<00:00, 73.26it/s]


21. #sobject=(54, 2), #tobject=(33, 2)
21. #sdatatype=(20, 2), #tdatatype=(0, 2)
CPU times: user 46min 16s, sys: 1min 3s, total: 47min 19s
Wall time: 7min 10s


In [45]:
all_refined_class_aligns_list[3]

,source,source_label_x,target,target_label_x,target_label_y,source_label_y,string_similarity,context_similarity,label_embeddings_similarity,label_embeddings_wd_similarity,string_context_similarity,label_emb_context_similarity,label_emb_wd_context_similarity,label_emb_wd_context_all_similarity
0,http://conference#Abstract,Abstract,http://sigkdd#Abstract,Abstract,Abstract,Abstract,1.0,0.957847,1.0,1.0,0.957847,0.957847,0.957847,0.957847
1,http://conference#Invited_speaker,Invited_speaker,http://sigkdd#Invited_Speaker,Invited_Speaker,Invited_Speaker,Invited_speaker,1.0,0.945618,1.0,1.0,0.945618,0.945618,0.945618,0.945618
2,http://conference#Organizing_committee,Organizing_committee,http://sigkdd#Organizing_Committee,Organizing_Committee,Organizing_Committee,Organizing_committee,1.0,0.863638,1.0,1.0,0.863638,0.863638,0.863638,0.863638
3,http://conference#Program_committee,Program_committee,http://sigkdd#Program_Committee,Program_Committee,Program_Committee,Program_committee,1.0,0.863590,1.0,1.0,0.863590,0.863590,0.863590,0.863590
4,http://conference#Committee,Committee,http://sigkdd#Committee,Committee,Committee,Committee,1.0,0.858325,1.0,1.0,0.858325,0.858325,0.858325,0.858325
5,http://conference#Review,Review,http://sigkdd#Review,Review,Review,Review,1.0,0.849671,1.0,1.0,0.849671,0.849671,0.849671,0.849671
6,http://conference#Paper,Paper,http://sigkdd#Paper,Paper,Paper,Paper,1.0,0.830518,1.0,1.0,0.830518,0.830518,0.830518,0.830518
7,http://conference#Conference,Conference,http://sigkdd#Conference,Conference,Conference,Conference,1.0,0.804346,1.0,1.0,0.804346,0.804346,0.804346,0.804346
8,http://conference#Person,Person,http://sigkdd#Person,Person,Person,Person,1.0,0.796165,1.0,1.0,0.796165,0.796165,0.796165,0.796165


In [47]:
len(all_refined_class_property_aligns_list)

21

In [48]:
pd.concat(all_class_property_evaluations_list).mean()

/tmp/ipykernel_71177/3735428356.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_property_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.380952
numOfPredicted              7.047619
precision                   0.920443
recall                      0.472735
f1                          0.607383
dtype: float64

In [32]:
pd.concat(all_class_evaluations_list).reset_index(drop=True)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case,similarity,similarity_threshold,similarity_cutoff,matching_pairing
0,15,12,13,0.923077,0.800000,0.857143,iasted-sigkdd,string_context_similarity,0.62,0.274400,one-to-one
1,7,4,5,0.800000,0.571429,0.666667,confof-sigkdd,string_context_similarity,0.62,0.109206,one-to-one
2,23,9,12,0.750000,0.391304,0.514286,edas-ekaw,string_context_similarity,0.62,0.619406,one-to-one
3,15,8,10,0.800000,0.533333,0.640000,conference-sigkdd,string_context_similarity,0.62,0.116958,one-to-one
4,19,8,9,0.888889,0.421053,0.571429,edas-iasted,string_context_similarity,0.62,0.265139,one-to-one
5,15,4,6,0.666667,0.266667,0.380952,cmt-conference,string_context_similarity,0.62,0.559763,one-to-one
6,4,4,6,0.666667,1.000000,0.800000,cmt-iasted,string_context_similarity,0.62,0.061687,one-to-one
7,16,4,5,0.800000,0.250000,0.380952,cmt-confof,string_context_similarity,0.62,0.527445,one-to-one
8,20,9,10,0.900000,0.450000,0.600000,confof-ekaw,string_context_similarity,0.62,0.316072,one-to-one
9,12,8,8,1.000000,0.666667,0.800000,cmt-sigkdd,string_context_similarity,0.62,0.571298,one-to-one


In [208]:
pd.concat(all_class_evaluations_list).mean()

/tmp/ipykernel_60200/3883584887.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_class_evaluations_list).mean()


numOfRefs                  14.523810
numOfCorrectlyPredicted     6.809524
numOfPredicted              8.047619
precision                   0.849485
recall                      0.503128
f1                          0.615739
similarity_threshold        0.620000
similarity_cutoff           0.357616
dtype: float64

In [209]:
pd.concat(all_object_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,15,0,1,0.000000,0.000000,0.000000,iasted-sigkdd
0,23,3,10,0.300000,0.130435,0.181818,edas-ekaw
0,15,0,4,0.000000,0.000000,0.000000,cmt-conference
0,12,1,1,1.000000,0.083333,0.153846,cmt-sigkdd
0,19,1,1,1.000000,0.052632,0.100000,confof-edas
0,11,3,4,0.750000,0.272727,0.400000,cmt-ekaw
0,9,0,2,0.000000,0.000000,0.000000,confof-iasted
0,15,0,1,0.000000,0.000000,0.000000,edas-sigkdd
0,17,0,5,0.000000,0.000000,0.000000,conference-edas
0,13,1,9,0.111111,0.076923,0.090909,cmt-edas


In [210]:
pd.concat(all_object_evaluations_list).mean()

/tmp/ipykernel_60200/3059542249.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_object_evaluations_list).mean()


numOfRefs                  15.818182
numOfCorrectlyPredicted     0.818182
numOfPredicted              3.727273
precision                   0.287374
recall                      0.056005
f1                          0.084234
dtype: float64

In [211]:
pd.concat(all_datatype_evaluations_list)

,numOfRefs,numOfCorrectlyPredicted,numOfPredicted,precision,recall,f1,test_case
0,7,1,2,0.500000,0.142857,0.222222,confof-sigkdd
0,15,1,4,0.250000,0.066667,0.105263,conference-sigkdd
0,19,0,1,0.000000,0.000000,0.000000,edas-iasted
0,15,1,4,0.250000,0.066667,0.105263,cmt-conference
0,16,2,3,0.666667,0.125000,0.210526,cmt-confof
0,12,1,3,0.333333,0.083333,0.133333,cmt-sigkdd
0,19,2,4,0.500000,0.105263,0.173913,confof-edas
0,15,3,3,1.000000,0.200000,0.333333,conference-confof
0,15,3,5,0.600000,0.200000,0.300000,edas-sigkdd
0,17,2,7,0.285714,0.117647,0.166667,conference-edas


In [212]:
pd.concat(all_datatype_evaluations_list).mean()

/tmp/ipykernel_60200/1550793241.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(all_datatype_evaluations_list).mean()


numOfRefs                  14.818182
numOfCorrectlyPredicted     1.454545
numOfPredicted              3.636364
precision                   0.398701
recall                      0.100676
f1                          0.159138
dtype: float64